<h1> Elo Merchant Category Recommendation :</h1>

Note :</br>
Baseline draft for submission is Kaggle Notebook ver_58 with best scores updated</br>
(https://github.com/samaujs/Data-Science/blob/main/Kaggle/sample/elo-eda-ml.ipynb)</br>

> Rank : 329 out of 4110 [Public Score : 3.68346] (8.00%) with Kaggle Notebook ver_57 (130 KSfeatures_opt2_blending)</br>
> Rank : 352 out of 4110 [Private Score : 3.61319] (8.56%) with Kaggle Notebook ver_31 (142 features)</br>

In [ ]:
! nvidia-smi -L

## **Import all the libraries :**

In [ ]:
import numpy as np
import pandas as pd
import warnings
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'
# plt.style.use('ggplot')

from scipy.stats import ks_2samp

import datetime
import seaborn as sns
sns.set_style("whitegrid")
import warnings 
warnings.simplefilter("ignore")

# For computing Variable Inflation Factor (VIF)
from statsmodels.stats.outliers_influence import variance_inflation_factor

# For memory garbage collection
import gc
# Use joblib instead of pickle to save a model in scikit-learn
import pickle
import joblib

# Display columns
pd.set_option('display.max_rows', 210)
pd.set_option('display.max_columns', 210)
pd.set_option('display.max_info_columns', 210)
print(pd.get_option("display.max_rows"), pd.get_option("display.max_columns"))
print(pd.get_option("display.max_info_rows"), pd.get_option("display.max_info_columns"))

## **Functions :**

In [ ]:
## Reference: https://www.kaggle.com/rinnqd/reduce-memory-usage
def reduce_memory_usage(df, verbose=True):
  '''
  This function reduces the memory sizes of dataframe by changing the datatypes of the columns.
  Parameters
  df - DataFrame whose size to be reduced
  verbose - Boolean, to mention the verbose required or not.
  '''
  numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
  start_mem = df.memory_usage().sum() / 1024**2
  for col in df.columns:
      col_type = df[col].dtypes
      if col_type in numerics:
          c_min = df[col].min()
          c_max = df[col].max()
          if str(col_type)[:3] == 'int':
              if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                  df[col] = df[col].astype(np.int8)
              elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                  df[col] = df[col].astype(np.int16)
              elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                  df[col] = df[col].astype(np.int32)
              elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                  df[col] = df[col].astype(np.int64)
          else:
              c_prec = df[col].apply(lambda x: np.finfo(x).precision).max()
              if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max and c_prec == np.finfo(np.float16).precision:
                  df[col] = df[col].astype(np.float16)
              elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max and c_prec == np.finfo(np.float32).precision:
                  df[col] = df[col].astype(np.float32)
              else:
                  df[col] = df[col].astype(np.float64)
  end_mem = df.memory_usage().sum() / 1024**2
  if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
  return df

# Check for missing values : train_data['new_hist_first_buy'].isna().any()
def check_missing_values(df):
    cols_missing_values = []
    for col in df.columns:
        if df[col].isna().any():
            cols_missing_values.append(col)
            print(col)
    return cols_missing_values
            
def create_new_columns(name, aggs):
    # get the individual key from dictionary and the corresponding list of functions for this key
    # For example : 'purchase_amount' key for functions ['sum','max','min','mean','var']
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

# Intersection function
def intersection(list1, list2): 
    return list(set(list1) & set(list2))

# Load saved model
def load_model_from_picklefile(filename):
    infile = open(filename,'rb')
    loaded_model = pickle.load(infile)
    infile.close()
    return loaded_model

# Save trained model
def save_model_to_picklefile(filename, save_model):
    model_file = open(filename,'wb')
    pickle.dump(save_model, model_file)
    model_file.close()

# Create file for submission to Kaggle
def create_file_for_submission(filename, card_ids, final_predictions):    
    kaggle = pd.DataFrame({'card_id': card_ids, 'target': final_predictions})
    kaggle.to_csv(filename, index=False)

In [ ]:
from gensim.models import Word2Vec

def word2vec_feature(prefix, df, groupby, target, size):
    df_bag = pd.DataFrame(df[[groupby, target]])
    df_bag[target] = df_bag[target].astype(str)
    df_bag[target].fillna('NAN', inplace=True)
    df_bag = df_bag.groupby(groupby, as_index=False)[target].agg({'list':(lambda x: list(x))}).reset_index()
    doc_list = list(df_bag['list'].values)
    
    w2v = Word2Vec(doc_list, vector_size=size, window=3, min_count=1, workers=32)
    vocab_keys = list(w2v.wv.index_to_key) # w2v.wv.vocab.keys() 
    w2v_array = []
    for v in vocab_keys :
        w2v_array.append(list(w2v.wv[v]))
    df_w2v = pd.DataFrame()
    df_w2v['vocab_keys'] = vocab_keys    
    df_w2v = pd.concat([df_w2v, pd.DataFrame(w2v_array)], axis=1)
    df_w2v.columns = [target] + ['w2v_%s_%s_%d'%(prefix,target,x) for x in range(size)]
    print ('df_w2v:' + str(df_w2v.shape))
    return df_w2v

In [ ]:
%%time
# Load data (Wall time: 1min 47s)
train_data = pd.read_csv('../input/elo-merchant-category-recommendation/train.csv')
test_data = pd.read_csv('../input/elo-merchant-category-recommendation/test.csv')
historical_data = pd.read_csv('../input/elo-merchant-category-recommendation/historical_transactions.csv')
newmerchant_data = pd.read_csv('../input/elo-merchant-category-recommendation/new_merchant_transactions.csv')
merchants_data = pd.read_csv('../input/elo-merchant-category-recommendation/merchants.csv')

In [ ]:
%%time
# Reduce memory usage of data (Wall time: 4min 25s)
train_data = reduce_memory_usage(train_data)
test_data = reduce_memory_usage(test_data)
historical_data = reduce_memory_usage(historical_data)
newmerchant_data = reduce_memory_usage(newmerchant_data)
merchants_data = reduce_memory_usage(merchants_data)

In [ ]:
train_data.shape, test_data.shape, historical_data.shape, newmerchant_data.shape, merchants_data.shape

# Part 1 : Building Machine Learning models

# **(A) Data Preprocessing and Feature Engineering (Exploratory Data Analysis)**
> #### With the performed EDA (refer to Kaggle Notebook version 2 for complete analysis), the following data manipulations can be carried out for building the machine learning models for the task.
> Eg. We observe from that the train and test datasets have similar distributions.
> They are all categorical variables
> feature_1, feature_2 and feature_3 has 5, 3 and 2 (binary) unique values respectively.

### Examining the feature statistics :

In [ ]:
# EDA : Check for numeric and non-numeric features
# numeric_feature_count = 0
# non_numeric_feature_count = 0
# numeric_features = []
# non_numeric_features = []

# for col in historical_data.columns:
#   if (historical_data[col].dtypes == 'object'):
#     non_numeric_feature_count += 1
#     non_numeric_features.append(col)
#     print("Non-numeric feature No. {} and name : {}".format(non_numeric_feature_count, col))
#     print("No. of Missing values : {}, Zero values : {} with Mode value : {}".\
#           format(historical_data[col].isnull().sum(), (historical_data[col] == 0).sum(),
#                  historical_data[col].mode().values[0]))
#     print("*" * 82)
#     print()
#   else:
#     numeric_feature_count += 1
#     numeric_features.append(col)
#     print("Numeric feature No. {} and name : {}".format(numeric_feature_count, col))
#     # (historical_data[col] == historical_data[col].median()).sum()
#     print("No. of Missing values : {}, Zero values : {} with Mode value : {}".\
#           format(historical_data[col].isnull().sum(), (historical_data[col] == 0).sum(),
#                  historical_data[col].mode().values[0])) # Get only the value without the index
#     print("-" * 82)
#     print()
    

# print("Total No. of Numeric features {} and Non-numeric features : {}".format(numeric_feature_count, non_numeric_feature_count))
# print("\nNumeric features : {} and \nNon-numeric features : {}".format(numeric_features, non_numeric_features))

In [ ]:
# EDA : There are multiple transactions a single 'card_id'
# for card_id[0], 'C_ID_0ab67a22ab' : there are 1304310 - 1304243 = 67 historical transactions
# for card_id[10], 'C_ID_4859ac9ed5' : there are 23622204 - 23622180 = 24 historical transactions
# historical_data.loc[23622180:23622204]

### Check for missing values in the loaded data frames

In [ ]:
check_missing_values(train_data), check_missing_values(test_data)

In [ ]:
check_missing_values(historical_data), check_missing_values(newmerchant_data), check_missing_values(merchants_data)

## Impute missing values (with mean, median, mode) :
* For historical and new merchant data with mode values

In [ ]:
# historical_data['category_2'].value_counts()
# historical_data['category_2'].unique() = [1., nan,  3.,  5.,  2.,  4.]
historical_data_cat2_mode = historical_data['category_2'].mode()
historical_data_merchant_id_mode = historical_data['merchant_id'].mode()
# historical_data['category_3'].value_counts()
historical_data_cat3_mode = historical_data['category_3'].mode()

print("historical_data mode for category_2 : {}".format(historical_data_cat2_mode[0]))
print("historical_data mode for merchant_id : {}".format(historical_data_merchant_id_mode[0]))
print("historical_data mode for category_3 : {}".format(historical_data_cat3_mode[0]))

# Replace missing values with mode values for category_3', 'merchant_id' and 'category_2'
# When inplace = True, the data is modified in place,
# which means it will return nothing and the dataframe is now updated.
for df in [historical_data, newmerchant_data]:
    df['category_2'].fillna(historical_data_cat2_mode[0], inplace = True)
    df['merchant_id'].fillna(historical_data_merchant_id_mode[0], inplace = True)
    df['category_3'].fillna(historical_data_cat3_mode[0], inplace = True)

## Observe outliers and replaced with median values in provided features :
* For historical and new merchant data with mode values

In [ ]:
# print('Quantile values for purchase amount in Historical Transaction :')
# print('25th Percentile :',historical_data['purchase_amount'].quantile(0.25))
# print('50th Percentile :',historical_data['purchase_amount'].quantile(0.50))
# print('75th Percentile :',historical_data['purchase_amount'].quantile(0.75))
# print('100th Percentile :',historical_data['purchase_amount'].quantile(1))
# print('\n******************************************************************\n')
# print('Quantile values for purchase amount in New Merchant Transaction :')
# print('25th Percentile :',newmerchant_data['purchase_amount'].quantile(0.25))
# print('50th Percentile :',newmerchant_data['purchase_amount'].quantile(0.50))
# print('75th Percentile :',newmerchant_data['purchase_amount'].quantile(0.75))
# print('100th Percentile :',newmerchant_data['purchase_amount'].quantile(1))

In [ ]:
# historical_data[historical_data['purchase_amount']  == 6010603.9717525]
historical_data_outlier_index = historical_data.loc[(historical_data['purchase_amount']  == 6010603.9717525)].index.values
print("The index of purchase amount in historical data :", historical_data_outlier_index)

# newmerchant_data[newmerchant_data['purchase_amount']  == 263.15749789]
newmerchant_data_outlier_index = newmerchant_data.loc[(newmerchant_data['purchase_amount']  == 263.15749789), 'purchase_amount'].index.values
print("The index of purchase amount in newmerchant data :", newmerchant_data_outlier_index)

print("Original 'Purchase amount' in historical data :", historical_data.loc[historical_data_outlier_index[0], 'purchase_amount'])
print("Original 'Purchase amount' in newmerchant data :", newmerchant_data.loc[newmerchant_data_outlier_index[0], 'purchase_amount'])

# Replace outlier 'purchase amount' with median in historical data
historical_data.loc[historical_data_outlier_index[0], 'purchase_amount'] = historical_data['purchase_amount'].median()
# Replace outlier 'purchase amount' with median in newmerchant data
newmerchant_data.loc[newmerchant_data_outlier_index[0], 'purchase_amount'] = newmerchant_data['purchase_amount'].median()

print("New median 'Purchase amount' in historical data :", historical_data.loc[historical_data_outlier_index[0], 'purchase_amount'])
print("New median 'Purchase amount' in newmerchant data :", newmerchant_data.loc[newmerchant_data_outlier_index[0], 'purchase_amount'])

## Impute missing value in test data :
* Replace with the earliest purchase date

In [ ]:
# test_data['first_active_month'].isna().value_counts()
missing_card_index = test_data.loc[test_data['first_active_month'].isna(), 'card_id'].index.values

print("The index of missing card in test data :", missing_card_index)
print("The missing cards in test data :\n", test_data.loc[test_data['first_active_month'].isna()])

In [ ]:
# missing_card_id = test_data.loc[test_data['first_active_month'].isna(), 'card_id'].reset_index(drop=True)[0]
# get the historical data for the missing card_id
# card_missing_first_active_month = historical_data.loc[historical_data['card_id'] == missing_card_id]

# print("Card_id : {} with {} transactions.".format(missing_card_id,
#                                                   card_missing_first_active_month.shape[0]))

# Replace with Earliest 'purchase_date' for card_id 'C_ID_c27b4f80f7' is '2017-03-09' or 
# Fill the missing value with the mode value '2017-09' in 'first_active_month' of test_data
# 55 transactions with card_id = 'C_ID_c27b4f80f7' that is without first_active_month

# format : YYYY-MM
test_data['first_active_month'].fillna('2017-03',inplace=True)

# test_data.loc[missing_card_index[0]]['first_active_month']
test_data.loc[test_data['card_id'] == 'C_ID_c27b4f80f7'] # 2017-09

In [ ]:
check_missing_values(train_data), check_missing_values(test_data)

* # Feature Engineering
* Aggregations (Historical, New Merchant and Merchant data)
* Feature Transformations (with mappings, data type, mean)
* Feature Engineering with feature creations and subset selections (from 4 to 151)

In [ ]:
# The purchase dates in historical_data from 2017-01-01 to 2018-02-28
# The purchase dates in newmerchant_data from 2017-03-01 to 2018-04-30

# historical_data['installments'].median() is 0.0
# newmerchant_data['installments'].median() is 1.0

* Generalization for 'purchase date' in 'historical data' and 'new merchant data'

In [ ]:
for df in [historical_data, newmerchant_data]:
    df_purchase_date_max = pd.to_datetime(df['purchase_date'].max())
    
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    #https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/73244
    
    # map category improves in ranking
    df['category_3'] = df['category_3'].map({'A':2, 'B':1, 'C':0})
    
    # 7 new columns for historical_data and newmerchant_data
    df['purchase_date_year'] = df['purchase_date'].dt.year
    df['purchase_date_weekofyear'] = df['purchase_date'].dt.weekofyear
    df['purchase_date_month'] = df['purchase_date'].dt.month
    df['purchase_date_dayofweek'] = df['purchase_date'].dt.dayofweek
    df['purchase_date_weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    df['purchase_date_hour'] = df['purchase_date'].dt.hour
    
    # Replace "datetime.datetime.today()" and apply floor division; which improves on the ranking
    df['purchase_date_month_diff'] = ((df_purchase_date_max - df['purchase_date']).dt.days)//30
    df['purchase_date_month_diff'] += df['month_lag']
    
    # Explore purchase_amount_sum_mean in 'category_1' which helps to improve on ranking
    # df['category_1_purchase_amount_sum'] = df.groupby(['category_1'])['purchase_amount'].transform('sum') # 'sum'
    # df['category_1_purchase_amount_mean'] = df.groupby(['category_1'])['purchase_amount'].transform('mean') # 'sum'
    
    # Replace 'installments' outliers and later replaced collectively with median for missing values
    df['installments'].replace(-1, np.nan, inplace=True) # 0.0 and 1.0 for historical_data and newmerchant_data respectively
    df['installments'].replace(999, np.nan, inplace=True)

historical_data['purchase_date'].head(5)

In [ ]:
# Additional 7 new columns for historical_data and newmerchant_data
historical_data.shape, newmerchant_data.shape, train_data.shape, test_data.shape

In [ ]:
check_missing_values(historical_data), check_missing_values(newmerchant_data)

## Create features with aggregations for 'purchase_amount'
- These are memory intensive operations; thus needs to be done before Word2Vec

In [ ]:
# Aggregation done within loop below will cause notebook to restart
# historical_data
# hist_sum_category_1_1_purchase_amount
target = 'purchase_amount'
col = 'category_1'
for cat1_unique_val in historical_data[col].unique():
    print("Creating {} column with category value {} with sum of {}".format(col, cat1_unique_val, target))
    print('card_sum_' + str(col) + '_' + str(cat1_unique_val) + '_' + str(target))
    print("-" * 80)
    historical_data['card_sum_' + str(col) + '_' + str(cat1_unique_val) + '_' + str(target)] =\
            historical_data['card_id'].map(historical_data[historical_data[col]==cat1_unique_val].groupby(['card_id'])[target].sum())

In [ ]:
# historical_data
# authorized_flag + purchase_amount
col = 'authorized_flag'
for authflag_unique_val in historical_data[col].unique():
    print("Creating {} column with authorized_flag value {} with sum of {}".format(col, authflag_unique_val, target))
    print('card_sum_' + str(col) + '_' + str(authflag_unique_val) + '_' + str(target))
    print("-" * 80)
    historical_data['card_sum_' + str(col) + '_' + str(authflag_unique_val) + '_' + str(target)] =\
        historical_data['card_id'].map(historical_data[historical_data[col]==authflag_unique_val].groupby(['card_id'])[target].sum())

In [ ]:
col = 'category_1'
# newmerchant_data
# category_1 + purchase_amount
for cat1_unique_val in newmerchant_data[col].unique():
    print("Creating {} column with category value {} with sum of {}".format(col, cat1_unique_val, target))
    print('card_sum_' + str(col) + '_' + str(cat1_unique_val) + '_' + str(target))
    print("-" * 80)
    newmerchant_data['card_sum_' + str(col) + '_' + str(cat1_unique_val) + '_' + str(target)] =\
            newmerchant_data['card_id'].map(newmerchant_data[newmerchant_data[col]==cat1_unique_val].groupby(['card_id'])[target].sum())

print("Shapes of historical_data and newmerchant_data after {} :".format(str(col) + '_' + str(target)))
print(historical_data.shape, newmerchant_data.shape)

# KS discard features 'card_sum_category_1_0_purchase_amount' later

In [ ]:
# newmerchant_data
# month_lag + purchase_amount
# col = 'month_lag'

# KS discard features 'card_sum_month_lag_1_purchase_amount', 'card_sum_month_lag_2_purchase_amount'
# for mthlag_unique_val in newmerchant_data[col].unique():
#     print("Creating {} column with month_lag value {} with sum of {}".format(col, mthlag_unique_val, target))
#     print('card_sum_' + str(col) + '_' + str(mthlag_unique_val) + '_' + str(target))
#     print("-" * 80)
#     newmerchant_data['card_sum_' + str(col) + '_' + str(mthlag_unique_val) + '_' + str(target)] =\
#             newmerchant_data['card_id'].map(newmerchant_data[newmerchant_data[col]==mthlag_unique_val].groupby(['card_id'])[target].sum())        
            
# print("Shapes of historical_data and newmerchant_data after {} :".format(str(col) + '_' + str(target)))
# print(historical_data.shape, newmerchant_data.shape)

In [ ]:
col = 'authorized_flag'
# newmerchant_data
# authorized_flag + purchase_amount
for authflag_unique_val in newmerchant_data[col].unique():
    print("Creating {} column with authorized_flag value {} with sum of {}".format(col, authflag_unique_val, target))
    print('card_sum_' + str(col) + '_' + str(authflag_unique_val) + '_' + str(target))
    print("-" * 80)
    newmerchant_data['card_sum_' + str(col) + '_' + str(authflag_unique_val) + '_' + str(target)] =\
        newmerchant_data['card_id'].map(newmerchant_data[newmerchant_data[col]==authflag_unique_val].groupby(['card_id'])[target].sum())

print("Shapes of historical_data and newmerchant_data after {} :".format(str(col) + '_' + str(target)))
print(historical_data.shape, newmerchant_data.shape)

# KS discard features 'card_sum_authorized_flag_1_purchase_amount' later

* # Word2Vec
- 'merchant_id' and 'merchant_category_id' in 'historical_data'
- 'merchant_category_id', 'merchant_id', 'city_id', 'state_id' and 'subsector_id' in 'newmerchant_data'

In [ ]:
%%time
# Wall time: 6min 2s
hist_merchant_id_w2v = word2vec_feature('hist', historical_data, 'card_id',  'merchant_id', 5) # category_3
hist_merchant_category_id_w2v = word2vec_feature('hist', historical_data, 'card_id',  'merchant_category_id', 5) # category_3
# hist_city_id_w2v = word2vec_feature('hist', historical_data, 'card_id',  'city_id', 5) # category_3

# w2v values vary in different runs
hist_merchant_id_w2v

In [ ]:
# Converting hist_merchant_category_id_w2v['merchant_category_id'] type from 'object' into 'int16'
hist_merchant_category_id_w2v['merchant_category_id'] = hist_merchant_category_id_w2v['merchant_category_id'].astype(np.int16)
# hist_city_id_w2v['city_id'] = hist_city_id_w2v['city_id'].astype(np.int16)

# hist_city_id_w2v['city_id'].dtype
hist_merchant_category_id_w2v['merchant_category_id'].dtype

In [ ]:
# Merging is memory-intensive with word2vec
historical_data = historical_data.merge(hist_merchant_category_id_w2v, on='merchant_category_id', how='left')
historical_data = historical_data.merge(hist_merchant_id_w2v, on='merchant_id', how='left')
historical_data.shape

In [ ]:
print("Data type of hist_merchant_id_w2v['merchant_id'] :", hist_merchant_id_w2v['merchant_id'].dtype)
del hist_merchant_id_w2v, hist_merchant_category_id_w2v # hist_city_id_w2v
gc.collect()

In [ ]:
# newmerchant_data[['merchant_category_id','merchant_id','city_id','state_id','subsector_id']].nunique()
# newmerchant_data[['merchant_category_id','merchant_id','city_id','state_id','subsector_id']].head(5)

In [ ]:
%%time
# Wall time: 2min 11s
# KS discard features
# new_merchant_city_id_w2v = word2vec_feature('new', newmerchant_data, 'card_id',  'city_id', 5)
# new_merchant_subsector_id_w2v = word2vec_feature('new', newmerchant_data, 'card_id',  'subsector_id', 5)
# new_merchant_id_w2v = word2vec_feature('new', newmerchant_data, 'card_id',  'merchant_id', 5) # category_3
# new_merchant_category_id_w2v = word2vec_feature('new', newmerchant_data, 'card_id',  'merchant_category_id', 5)
new_merchant_state_id_w2v = word2vec_feature('new', newmerchant_data, 'card_id',  'state_id', 5) # category_3

# w2v values vary in different runs
new_merchant_state_id_w2v

In [ ]:
# new_merchant_id_w2v['merchant_id'] = new_merchant_id_w2v['merchant_id'].astype(np.int16)
# newmerchant_data['merchant_id'].head(2)

# new_merchant_id_w2v['merchant_id'].dtype, new_merchant_category_id_w2v['merchant_category_id'].dtype,\
# new_merchant_city_id_w2v['city_id'].dtype, new_merchant_state_id_w2v['state_id'].dtype,\
# new_merchant_subsector_id_w2v['subsector_id'].dtype
historical_data.shape, newmerchant_data.shape

In [ ]:
# new_merchant_city_id_w2v['city_id'] = new_merchant_city_id_w2v['city_id'].astype(np.int16)
# new_merchant_subsector_id_w2v['subsector_id'] = new_merchant_subsector_id_w2v['subsector_id'].astype(np.int16)
# new_merchant_id_w2v['merchant_id'] = new_merchant_id_w2v['merchant_id'].astype(str)
# new_merchant_category_id_w2v['merchant_category_id'] = new_merchant_category_id_w2v['merchant_category_id'].astype(np.int16)
new_merchant_state_id_w2v['state_id'] = new_merchant_state_id_w2v['state_id'].astype(np.int16)

# new_merchant_id_w2v['merchant_id'].dtype, new_merchant_category_id_w2v['merchant_category_id'].dtype,\
# new_merchant_city_id_w2v['city_id'].dtype, new_merchant_subsector_id_w2v['subsector_id'].dtype, \
new_merchant_state_id_w2v['state_id'].dtype

In [ ]:
# newmerchant_data = newmerchant_data.merge(new_merchant_city_id_w2v, on='city_id', how='left')
# newmerchant_data = newmerchant_data.merge(new_merchant_subsector_id_w2v, on='subsector_id', how='left')
# newmerchant_data = newmerchant_data.merge(new_merchant_id_w2v, on='merchant_id', how='left')
# newmerchant_data = newmerchant_data.merge(new_merchant_category_id_w2v, on='merchant_category_id', how='left')
newmerchant_data = newmerchant_data.merge(new_merchant_state_id_w2v, on='state_id', how='left')

historical_data.shape, newmerchant_data.shape

In [ ]:
# del new_merchant_city_id_w2v, new_merchant_subsector_id_w2v, new_merchant_id_w2v, new_merchant_category_id_w2v
del new_merchant_state_id_w2v
gc.collect()

## Aggregate 'historical_data' with 'card_id'

In [ ]:
historical_data['month_lag'].value_counts()
# historical_data['category_1'].value_counts()

In [ ]:
%%time
aggs = {}

# 'dayofweek', 'year'
for col in ['purchase_date_month', 'purchase_date_hour', 'purchase_date_weekofyear', 'subsector_id', 'merchant_id']: # 'merchant_category_id', 'month','hour','weekofyear'
            # 'state_id', 'city_id']: # added more features
    aggs[col] = ['nunique']

# KS discard feature 'hist_purchase_amount_max', keep to create other ratio features that are useful
aggs['purchase_amount'] = ['sum', 'min', 'max', 'mean', 'std', 'median'] # check 'std' instead of var'
# KS discard feature 'hist_installments_mean'
aggs['installments'] = ['sum', 'max', 'mean', 'std'] # 'median', 'min', 'mean', check 'std' instead of var'
aggs['purchase_date'] = ['min', 'max'] # peak-to-peak (maximum - minimum)
aggs['month_lag'] = ['min', 'mean', 'std'] # ['max','min','mean','var'], check 'std' instead of var'
aggs['authorized_flag'] = ['sum', 'mean']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

aggs['purchase_date_weekend'] = ['sum', 'mean']
aggs['purchase_date_month_diff'] = ['mean']

# Check features if can achieve better performance
# These features did not help to ascend the rank climb
# aggs['purchase_date_year'] = ['nunique', 'sum', 'mean']
# aggs['purchase_date_weekofyear'] = ['nunique', 'sum', 'mean']
# aggs['purchase_date_month'] = ['nunique', 'sum', 'mean']
# aggs['purchase_date_dayofweek'] = ['nunique', 'sum', 'mean']
# aggs['purchase_date_hour'] = ['nunique', 'sum', 'mean']

# Check features if can achieve better performance
aggs['merchant_category_id'] = ['nunique', 'std']

# Can try by simply make a copy without aggregations
aggs['card_sum_category_1_0_purchase_amount'] = ['sum'] # 'std', 'mean'
aggs['card_sum_category_1_1_purchase_amount'] = ['sum'] # 'std', 'mean'

# 'hist_card_sum_authorized_flag_0_purchase_amount_mean'
aggs['card_sum_authorized_flag_0_purchase_amount'] = ['sum'] # 'std', 'mean'
aggs['card_sum_authorized_flag_1_purchase_amount'] = ['mean', 'sum'] # 'std'

aggs['w2v_hist_merchant_category_id_0'] = ['std']
aggs['w2v_hist_merchant_category_id_1'] = ['std']
aggs['w2v_hist_merchant_category_id_2'] = ['std']
aggs['w2v_hist_merchant_category_id_3'] = ['std']
aggs['w2v_hist_merchant_category_id_4'] = ['std']

aggs['w2v_hist_merchant_id_0'] = ['std']
aggs['w2v_hist_merchant_id_1'] = ['std']
# KS discard features
# 'hist_w2v_hist_merchant_id_2_std', 'hist_w2v_hist_merchant_id_3_std'
aggs['w2v_hist_merchant_id_2'] = ['std']
aggs['w2v_hist_merchant_id_3'] = ['std']
aggs['w2v_hist_merchant_id_4'] = ['std']


# Explore purchase_amount_sum_mean
# aggs['category_1_purchase_amount_sum'] = ['mean', 'sum']
# aggs['category_1_purchase_amount_mean'] = ['mean', 'sum']

# added more features with median for 'installments' and ptp for 'purchase_date'
# aggs['month'] = [ 'min', 'max', 'mean', 'var']

# KS discard features
# 'hist_category_2_purchase_amount_mean_mean', 'hist_category_3_purchase_amount_mean_mean',
# 'hist_category_3_purchase_amount_sum_mean',
# Create 'purchase_amount_mean' for the groupby columns
# Include 'category_1' for groupby sum
for col in ['category_1', 'category_2', 'category_3']:
    historical_data[col+'_sum_purchase_amount'] = historical_data.groupby([col])['purchase_amount'].transform('sum') # 'mean'
    # aggs[col+'_purchase_amount_mean'] = ['mean']
    aggs[col+'_sum_purchase_amount'] = ['sum', 'mean', 'std']


# Eg: 'hist_month_nunique', 'hist_hour_nunique'
new_columns = create_new_columns('hist',aggs)
print("New aggregated columns :\n", new_columns)

# Perform the aggregations with 'card_id'
df_hist_trans_group = historical_data.groupby('card_id').agg(aggs)

df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)

print("Constructing features without aggregations ...")
# Create features without aggregations 
df_hist_trans_group['hist_purchase_date_diff'] = (df_hist_trans_group['hist_purchase_date_max'] - df_hist_trans_group['hist_purchase_date_min']).dt.days
df_hist_trans_group['hist_purchase_date_average'] = df_hist_trans_group['hist_purchase_date_diff']/df_hist_trans_group['hist_card_id_size']
df_hist_trans_group['hist_purchase_date_uptonow'] = (datetime.datetime.today() - df_hist_trans_group['hist_purchase_date_max']).dt.days

# Explore new features for rank ascend
df_hist_trans_group['hist_card_last_woy'] = df_hist_trans_group['hist_purchase_date_max'].dt.weekofyear
df_hist_trans_group['hist_card_last_doy'] = df_hist_trans_group['hist_purchase_date_max'].dt.dayofyear
df_hist_trans_group['hist_card_last_day'] = df_hist_trans_group['hist_purchase_date_max'].dt.day

# --
print("-" * 80)
# References :
# https://www.kaggle.com/raddar/target-true-meaning-revealed
# https://www.kaggle.com/raddar/towards-de-anonymizing-the-data-some-insights
target = 'hist_purchase_amount_new'
print("\nConstructing : 'hist_card_sum_purchase_amount_new'")
df_hist_trans_group['hist_purchase_amount_new'] = np.round((historical_data['purchase_amount'] / 0.00150265118) + 497.06, 2)
# Checkout new feature for effects on RMSE
df_hist_trans_group['hist_card_sum_purchase_amount_new'] = df_hist_trans_group['card_id'].map(df_hist_trans_group.groupby(['card_id'])[target].sum())
df_hist_trans_group['hist_card_mean_purchase_amount_new'] = df_hist_trans_group['card_id'].map(df_hist_trans_group.groupby(['card_id'])[target].mean())
df_hist_trans_group['hist_card_median_purchase_amount_new'] = df_hist_trans_group['card_id'].map(df_hist_trans_group.groupby(['card_id'])[target].median())
df_hist_trans_group['hist_card_min_purchase_amount_new'] = df_hist_trans_group['card_id'].map(df_hist_trans_group.groupby(['card_id'])[target].min())
df_hist_trans_group['hist_card_max_purchase_amount_new'] = df_hist_trans_group['card_id'].map(df_hist_trans_group.groupby(['card_id'])[target].max())
df_hist_trans_group['hist_card_std_purchase_amount_new'] = df_hist_trans_group['card_id'].map(df_hist_trans_group.groupby(['card_id'])[target].std())

# 'hist_card_sum_month_lag_0_purchase_amount_new'
# target = 'hist_card_sum_purchase_amount_new'
# print("Constructing : 'hist_card_sum_month_lag_x_purchase_amount_new'")
# df_hist_trans_group['hist_month_lag'] = historical_data['month_lag']
# KS discard feature 'hist_card_sum_month_lag_0_purchase_amount_new', 'hist_card_sum_month_lag_neg2_purchase_amount_new'
# Replace df_hist_trans_group with historical_data will cause insufficient memories in the notebook
# df_hist_trans_group['hist_card_sum_month_lag_neg2_purchase_amount_new'] = \
#     df_hist_trans_group['card_id'].map(df_hist_trans_group[df_hist_trans_group['hist_month_lag']==-2].groupby(['card_id'])[target].sum())

# Check which variation has higher importance ranking
print("Copying : variations of 'category_1'")
# card_sum_category_1_x_purchase_amount
df_hist_trans_group['hist_card_sum_category_1_0_purchase_amount'] = historical_data['card_sum_category_1_0_purchase_amount']

# KS discard feature 'hist_card_sum_category_1_1_purchase_amount'
df_hist_trans_group['hist_card_sum_category_1_1_purchase_amount'] = historical_data['card_sum_category_1_1_purchase_amount']
# 'hist_sum_category_1_1_purchase_amount' generated above with separate columns of aggregrations
df_hist_trans_group['hist_sum_category_1_purchase_amount'] = historical_data['category_1_sum_purchase_amount']

# These features did not help to ascend the rank climb
target = 'purchase_amount'
print("Constructing : 'hist_sum_category_1_1_purchase_amount'")
df_hist_trans_group['hist_category_1'] = historical_data['category_1']
df_hist_trans_group['purchase_amount'] = historical_data['purchase_amount']
df_hist_trans_group['hist_sum_category_1_1_purchase_amount'] = \
    df_hist_trans_group['hist_category_1'].map(df_hist_trans_group[df_hist_trans_group['hist_category_1']==1].groupby(['hist_category_1'])[target].sum())

# Explore new building feature, 'card_purchase_date_max_ratio', for rank ascend
df_hist_trans_group['hist_purchase_date_int'] = historical_data['purchase_date'].astype(np.int64) * 1e-9
df_hist_trans_group['hist_card_purchase_date_max'] = df_hist_trans_group.groupby(['card_id'])['hist_purchase_date_int'].transform('max')
# --

print()
print("Shape of df_hist_trans_group :\n", df_hist_trans_group.shape)
print("df_hist_trans_group columns :\n", df_hist_trans_group.columns)


df_train = train_data.merge(df_hist_trans_group, on='card_id',how='left')
df_test = test_data.merge(df_hist_trans_group, on='card_id',how='left')

# (325540, 60) in v50 to (325540, 76) in v54
# Features with low importances
# 'hist_sum_category_1_1_purchase_amount','hist_card_mean_purchase_amount_new',
# 'hist_card_sum_purchase_amount_new', 'hist_card_min_purchase_amount_new',
# 'hist_category_1', 'hist_sum_category_1_purchase_amount', 
# 'new_merchant_card_min_purchase_amount_new', 'new_merchant_card_sum_purchase_amount_new',
# 'new_merchant_purchase_amount_new', 'new_merchant_card_median_purchase_amount_new'

In [ ]:
# historical_data['month_lag'].nunique() gives 14
# historical_data (out of memory), could try within loop above by just copying the needed column
# month_lag + purchase_amount
# col = 'month_lag'

# for mthlag_unique_val in historical_data[col].unique():
#     print("Creating {} column with month_lag value {} with sum of {}".format(col, mthlag_unique_val, target))
#     print('card_sum_' + str(col) + '_' + str(mthlag_unique_val) + '_' + str(target))
#     print("-" * 80)
#     historical_data['card_sum_' + str(col) + '_' + str(mthlag_unique_val) + '_' + str(target)] =\
#             historical_data['card_id'].map(historical_data[historical_data[col]==mthlag_unique_val].groupby(['card_id'])[target].sum())        
            
# print("Shapes of historical_data and newmerchant_data after {} :".format(str(col) + '_' + str(target)))
# print(historical_data.shape, newmerchant_data.shape)

In [ ]:
# After merging with historical data with 'left' with group by card_id
print(df_hist_trans_group.shape, df_train.shape, df_test.shape)

# Release memory
del df_hist_trans_group, train_data, test_data
gc.collect()

## Create more aggregated features from 'hist_purchase_amount_new'

In [ ]:
# Check for missing values
# train_data['new_hist_first_buy'].isna().any()
# check_missing_values(df_train), check_missing_values(df_test)

In [ ]:
# ((201917, 50), (123623, 49))
df_train.shape, df_test.shape, historical_data.shape, newmerchant_data.shape
# df_train.info()

#### Exploring New Merchant data

In [ ]:
# historical_data['category_1_purchase_amount_sum'], newmerchant_data['category_1_purchase_amount_sum']
# check_missing_values(newmerchant_data), check_missing_values(merchants_data)

# There are no duplicate card_ids in train_data with 201,917 transactions
# train_data_cards = train_data['card_id']
# train_data_cards.nunique()

# test_data_cards = test_data['card_id']
# There are no duplicate card_ids in test_data with 123,623 transactions
# test_data_cards.nunique()

# print("Total number of unique cards in train_data and test_data :", (train_data_cards.nunique() + test_data_cards.nunique()))

# historical_data_cards = historical_data['card_id']
# The unique card_ids in train_data and test_date makes up the unique 325,540 card_ids in historical_data
# historical_data_cards.nunique()

#### Note :
> There are NO intersected values between train and test sets

> All unique card_ids in train_data and test_data can be found in historical_data

> All unique card_ids in train_data and test_data can be found in newmerchant_data

In [ ]:
# Total number of unique cards in train_data and test_data = Total of unique cards in historical_data
# intersect_list = intersection(train_data_cards, test_data_cards)
# print("There are NO intersected card_id :", intersect_list)

# np.intersect1d(train_data_cards, test_data_cards)
# intersect_list = list(set(train_data_cards) & set(test_data_cards))
# print("No. of intersected card_ids :", len(intersect_list))
# --
# pandas.core.series.Series
# s1 = pd.Series([4, 5, 20, 42, 42, 43])
# s2 = pd.Series([1, 2, 3, 5, 42])
# set(s1)&set(s2) same results
# np.intersect1d(s1, s2)
# --
# Difference in card_ids
# card_difference = set(train_data_cards).symmetric_difference(test_data_cards)
# print("Total no. of card_ids in train_data and test_data which are unique : {}".format(len(card_difference)))
# --
# union_cards_df = pd.concat([train_data_cards, test_data_cards])

# type(union_cards_df) is pandas.core.series.Series
# union_data_size = train_data_cards.nunique() + test_data_cards.nunique() = 325,540
# The card_ids in historical_data intersects with all the card_ids found in train and test sets
# intersect_list = intersection(historical_data_cards, union_cards_df)
# len(intersect_list)
# --
# card_difference = set(historical_data_cards).symmetric_difference(union_cards_df)
# print("The card_ids in train and test but NOT found in historical_data : {}".format(len(card_difference)))
# --
# union_data_size = train_data_cards.nunique() + test_data_cards.nunique() = 325,540
# Union returns a new set with elements from the set and the specified iterables
# card_union = set(train_data_cards).union(test_data_cards)
# print("Total no. of unique cards : ", len(card_union))

# symmetric_difference returns a new set with elements in either the set or the specified iterable but not both.
# The card_ids in historical_data can be found in train and test sets; there are no difference in card_ids
# card_difference = set(historical_data_cards).symmetric_difference(union_cards_df)
# card_difference = card_union.symmetric_difference(historical_data_cards)
# print("Unique cards of train and test data but not in historical data : ", len(card_difference))
# --
# newmerchant_data_cards = newmerchant_data['card_id']
# There are 290,001 unique cards in newmerchant_data but train_data has 201,917 cards
# print("No. of unique cards in train_data :", train_data_cards.nunique())
# print("No. of unique cards in newmerchant_data :", newmerchant_data_cards.nunique())

# intersect_list_train_newmerchant = intersection(train_data_cards, newmerchant_data_cards)
# print("No. of intersected card_ids between train_data_cards and newmerchant_data_cards :", len(intersect_list_train_newmerchant))

# These cards create NaNs during merging
# print("Unique card_ids in train_data but not in the newmerchant_data : ",
#       train_data_cards.nunique() - len(intersect_list_train_newmerchant))
# --
# card_difference = set(train_data_cards).symmetric_difference(newmerchant_data_cards)
# print("Unique card_ids not in the intersection of train_data and the newmerchant_data : ", len(card_difference))
# --
# There are 290,001 unique cards in newmerchant_data but train_data has 201,917 cards
# print("No. of unique cards in test_data :", test_data_cards.nunique())
# print("No. of unique cards in newmerchant_data :", newmerchant_data_cards.nunique())

# intersect_list_test_newmerchant = intersection(test_data_cards, newmerchant_data_cards)
# print("No. of intersected card_ids between test_data_cards and newmerchant_data_cards :", len(intersect_list_test_newmerchant))

# These cards create NaNs during merging
# print("Unique card_ids in test_data but not in the newmerchant_data : ",
#       test_data_cards.nunique() - len(intersect_list_test_newmerchant))
# --
# train_test_newmerchant_card_union = set(intersect_list_train_newmerchant).union(intersect_list_test_newmerchant)
# len(intersect_list_train_newmerchant) + len(intersect_list_test_newmerchant) gives same result of 290,001
# len(train_test_newmerchant_card_union)
# --
# card_difference = train_test_newmerchant_card_union.symmetric_difference(newmerchant_data_cards)
# print("Verify that intersected unique cards of train and test data with newmerchant data but not in newmerchant data : ",
#       len(card_difference))

#### Exploring Merchant data

In [ ]:
merchants_data_ids = merchants_data['merchant_id']
newmerchants_data_ids = newmerchant_data['merchant_id']

# There are 334,633 merchant_id in merchants_data but newmerchant_data has only 226,129 merchant_id
print("No. of unique merchant_id in merchants_data :", merchants_data_ids.nunique())
print("No. of unique merchant_id in newmerchant_data :", newmerchants_data_ids.nunique())

intersect_list_newmerchants_merchants = intersection(newmerchants_data_ids, merchants_data_ids)
print("No. of intersected merchant_ids between newmerchants_data_ids and merchants_data_ids :", len(intersect_list_newmerchants_merchants))

# All merchant_id in newmerchant_data can be found in merchants_data
print("Unique merchant_ids in newmerchants_data but not in the merchants_data :",
      newmerchants_data_ids.nunique() - len(intersect_list_newmerchants_merchants))

In [ ]:
merchants_data_ids.value_counts()

In [ ]:
merchants_data_cat_ids = merchants_data['merchant_category_id']
newmerchants_data_cat_ids = newmerchant_data['merchant_category_id']

# There are 334,633 merchant_id in merchants_data but newmerchant_data has only 226,129 merchant_id
print("No. of unique merchant_category_id in merchants_data :", merchants_data_cat_ids.nunique())
print("No. of unique merchant_category_id in newmerchant_data :", newmerchants_data_cat_ids.nunique())

intersect_list_newmerchants_merchants_cat_id = intersection(newmerchants_data_cat_ids, merchants_data_cat_ids)
print("No. of intersected merchant_category_ids between newmerchants_data_cat_ids and merchants_data_cat_ids :",
      len(intersect_list_newmerchants_merchants_cat_id))

# These merchant_category_id create NaNs during merging
print("Unique merchant_category_ids in newmerchants_data but not in the merchants_data :",
      newmerchants_data_cat_ids.nunique() - len(intersect_list_newmerchants_merchants_cat_id))

In [ ]:
merchants_data_ids = merchants_data['merchant_id']
historical_data_merchant_ids = historical_data['merchant_id']

# There are 334,633 merchant_id in merchants_data but newmerchant_data has only 226,129 merchant_id
print("No. of unique merchant_id in merchants_data :", merchants_data_ids.nunique())
print("No. of unique merchant_id in historical_data :", historical_data_merchant_ids.nunique())

intersect_list_historical_merchants = intersection(historical_data_merchant_ids, merchants_data_ids)
print("No. of intersected merchant_ids between historical_data_merchant_ids and merchants_data_ids :",
      len(intersect_list_historical_merchants))

# All merchant_id in historical_data can be found in merchants_data
print("Unique merchant_ids in historical_data but not in the merchants_data :",
      historical_data_merchant_ids.nunique() - len(intersect_list_historical_merchants))

In [ ]:
merchants_data_cat_ids = merchants_data['merchant_category_id']
historical_data_cat_ids = historical_data['merchant_category_id']

# There are 334,633 merchant_id in merchants_data but newmerchant_data has only 226,129 merchant_id
print("No. of unique merchant_category_id in merchants_data :", merchants_data_cat_ids.nunique())
print("No. of unique merchant_category_id in historical_data :", historical_data_cat_ids.nunique())

intersect_list_historical_merchants_cat_id = intersection(historical_data_cat_ids, merchants_data_cat_ids)
print("No. of intersected merchant_category_ids between historical_data_cat_ids and merchants_data_cat_ids :",
      len(intersect_list_historical_merchants_cat_id))

# These merchant_category_id create NaNs during merging
print("Unique merchant_category_ids in historical_data but not in the merchants_data :",
      historical_data_cat_ids.nunique() - len(intersect_list_historical_merchants_cat_id))

print()
print("There are MORE merchant_category_ids in historical_data than in the merchants_data :",
      (historical_data_cat_ids.nunique() - merchants_data_cat_ids.nunique()))

In [ ]:
# merchants_data['merchant_id'].describe()
# df_train.shape, df_test.shape, newmerchant_data.shape, merchants_data.shape
# check_missing_values(newmerchant_data), check_missing_values(merchants_data)

# There are duplicate values of 'merchant_id'
# merchants_data['merchant_id'].describe()

## Impute missing values in merchants data with respective mode values

In [ ]:
# print(merchants_data['avg_sales_lag3'].isna().value_counts())
# print()
# print("Mode value of avg_sales_lag3 :", merchants_data['avg_sales_lag3'].mode()[0])

# missing_avg_sales_l3_index = merchants_data.loc[merchants_data['avg_sales_lag3'].isna(), 'merchant_id'].index.values
# print("The index of missing avg_sales_lag3 in merchants_data :", missing_avg_sales_l3_index)

# merchants_data.loc[merchants_data['avg_sales_lag3'].isna()]
# --
# print(merchants_data['avg_sales_lag6'].isna().value_counts())
# print()
# print("Mode value of avg_sales_lag6 :", merchants_data['avg_sales_lag6'].mode()[0])

# missing_avg_sales_l6_index = merchants_data.loc[merchants_data['avg_sales_lag6'].isna(), 'merchant_id'].index.values
# print("The index of missing avg_sales_lag6 in merchants_data :", missing_avg_sales_l6_index)

# merchants_data.loc[merchants_data['avg_sales_lag6'].isna()]
# --
# print(merchants_data['avg_sales_lag12'].isna().value_counts())
# print()
# print("Mode value of avg_sales_lag12 :", merchants_data['avg_sales_lag12'].mode()[0])

# missing_avg_sales_l12_index = merchants_data.loc[merchants_data['avg_sales_lag12'].isna(), 'merchant_id'].index.values
# print("The index of missing avg_sales_lag12 in merchants_data :", missing_avg_sales_l12_index)

# merchants_data.loc[merchants_data['avg_sales_lag12'].isna()]
# --
# print(merchants_data['category_2'].isna().value_counts())
# print()
# print("Mode value of category_2 :", merchants_data['category_2'].mode()[0])

# missing_category_2_index = merchants_data.loc[merchants_data['category_2'].isna(), 'merchant_id'].index.values
# print("The index of missing category_2 in merchants_data :", missing_category_2_index)

# merchants_data.loc[missing_category_2_index]

In [ ]:
# Fill missing values in merchants_data with respective mode values
merchants_data['avg_sales_lag3'].fillna(merchants_data['avg_sales_lag3'].mode()[0], inplace = True)
merchants_data['avg_sales_lag6'].fillna(merchants_data['avg_sales_lag6'].mode()[0], inplace = True)
merchants_data['avg_sales_lag12'].fillna(merchants_data['avg_sales_lag12'].mode()[0], inplace = True)
merchants_data['category_2'].fillna(merchants_data['category_2'].mode()[0], inplace = True)
merchants_data.loc[3:15]

In [ ]:
check_missing_values(merchants_data), merchants_data.shape, newmerchant_data.shape

## Merging strategy :
> 1. Aggregate historical data with ***'card_id'*** and merge to train and test with groupby ***'card_id'***
> 2. Aggregate merchant data with ***'merchant_id'*** and merge to new merchant data with groupby ***'merchant_id'***
> 3. Merge new merchant data with merchant data to new train and test with groupby ***'card_id'***

In [ ]:
# merchants_data['most_recent_sales_range'].value_counts()

# merchants_data['most_recent_purchases_range'].value_counts()
# merchants_data['most_recent_purchases_range'].min(), merchants_data['most_recent_purchases_range'].max()

# merchants_data['most_recent_sales_range'].mode()

# aggs = {}
# aggs['most_recent_sales_range'] = ['min']
# md_tmp = merchants_data.groupby('merchant_id').agg(aggs)
# md_tmp.shape

# md_tmp = pd.DataFrame()
# Group based on 'purchase_amount'
# for col in ['most_recent_sales_range','most_recent_purchases_range']:
#     md_tmp[col+'_avg_sales_lag3_mean'] = merchants_data.groupby([col])['avg_sales_lag3'].transform('min')
#     md_tmp[col+'_avg_sales_lag6_mean'] = merchants_data.groupby([col])['avg_sales_lag6'].transform('min')
#     md_tmp[col+'_avg_sales_lag12_mean'] = merchants_data.groupby([col])['avg_sales_lag12'].transform('min')

# md_tmp.reset_index(drop=False,inplace=True)
# md_tmp.head(2)
# check_missing_values(md_tmp)

## Feature mappings in merchants_data

In [ ]:
sales_purchases_range_dict= {'A':5, 'B':4, 'C':3, 'D':2, 'E':1}

for df in [merchants_data]:
    df['most_recent_sales_range'] = df['most_recent_sales_range'].map(sales_purchases_range_dict)
    df['most_recent_purchases_range'] = df['most_recent_purchases_range'].map(sales_purchases_range_dict)
    # df['category_1'] = df['category_1'].map({'Y':1, 'N':0})

In [ ]:
# Majority revenue and quantity of transactions in last active month are high in merchants data
# df['most_recent_sales_range'].value_counts(), df['most_recent_purchases_range'].value_counts(), df['category_1'].value_counts()

# merchants_data['merchant_id'].nunique()

# merchants_data['city_id'].describe()

## Aggregate 'merchants_data' with 'merchant_id'

In [ ]:
aggs = {}

for col in ['subsector_id', 'category_4','city_id', 'state_id']: # 'merchant_id', 'merchant_group_id'
    aggs[col] = ['nunique']

aggs['numerical_1'] = ['sum', 'mean']
# KS discard feature 'merchants_numerical_2_sum'
aggs['numerical_2'] = ['mean'] # 'sum'
aggs['avg_sales_lag3'] = ['sum', 'mean', 'size', 'min', 'max']
# KS discard feature 'merchants_avg_purchases_lag3_min'
aggs['avg_purchases_lag3'] = ['sum', 'mean', 'size', 'max'] # 'min'
aggs['active_months_lag3'] = ['sum', 'mean', 'size', 'min', 'max']
aggs['avg_sales_lag6'] = ['sum', 'mean', 'size', 'min', 'max']
# KS discard feature 'merchants_avg_purchases_lag6_sum'
aggs['avg_purchases_lag6'] = ['mean', 'size', 'min', 'max'] # 'sum'
aggs['active_months_lag6'] = ['sum', 'mean', 'size', 'min', 'max']
aggs['avg_sales_lag12'] = ['sum', 'mean', 'size', 'min', 'max']
# KS discard feature 'merchants_avg_purchases_lag12_sum', 'merchants_avg_purchases_lag12_min'
aggs['avg_purchases_lag12'] = ['mean', 'size', 'max'] # 'sum', 'min'
aggs['active_months_lag12'] = ['sum', 'mean', 'size', 'min', 'max']
aggs['category_2'] = ['mean']

# Explore more features
aggs['most_recent_sales_range'] = ['min', 'max'] 
aggs['most_recent_purchases_range'] = ['min', 'max']

# Removed for low feaature importance
# aggs['merchant_category_id'] = ['std']

# aggs['merchant_category_id'] = ['sum', 'mean', 'size', 'min', 'max']

# 'category_1_nunique' is not created even if added above with the other col
# aggs['category_1'] = ['min', 'max', 'nunique']

new_columns = create_new_columns('merchants',aggs)
print("New columns :\n", new_columns)
df_merchant_id_group = merchants_data.groupby('merchant_id').agg(aggs) # merchant_category_id

df_merchant_id_group.columns = new_columns
df_merchant_id_group.reset_index(drop=False, inplace=True)

print("Shape of df_merchant_category :\n", df_merchant_id_group.shape)
print("df_merchant_id_group columns :\n", df_merchant_id_group.columns)
df_merchant_id_group

# Remove aggs['merchant_category_id'] = ['std'] in v54

In [ ]:
# There are missing values in 'merchants_merchant_category_id_std'
# df_merchant_id_group['merchants_merchant_category_id_std'].isna().value_counts()
# True     334592
# False        41

In [ ]:
# %%time
# hist_merchant_id_w2v = word2vec_feature('hist', historical_data, 'card_id',  'merchant_id', 5) # category_3
# hist_merchant_id_w2v
# --
# %%time
# hist_merchant_category_id_w2v = word2vec_feature('hist', historical_data, 'card_id',  'merchant_category_id', 5) # category_3
# hist_merchant_category_id_w2v
# --
# merchants_data.shape, historical_data.nunique()
# --
# merchants_data = merchants_data.merge(hist_merchant_id_w2v, on='merchant_id', how='left') # df_merchant_id_group
# --
# # Converting hist_merchant_category_id_w2v['merchant_category_id'] type from 'object' into 'int16'
# hist_merchant_category_id_w2v['merchant_category_id'] = hist_merchant_category_id_w2v['merchant_category_id'].astype(np.int16)
# hist_merchant_category_id_w2v['merchant_category_id'].dtype
# --
# merchants_data = merchants_data.merge(hist_merchant_category_id_w2v, on='merchant_category_id', how='left')
# merchants_data.shape
# --

In [ ]:
# Group newmerchant_data and merchants_data
newmerchant_with_merchants_data = newmerchant_data.merge(df_merchant_id_group, on='merchant_id',how='left') # merchant_category_id
newmerchant_with_merchants_data.shape

In [ ]:
df_train.shape, df_test.shape, newmerchant_data.shape, merchants_data.shape, df_merchant_id_group.shape

#### When merging with 'merchant_category_id', there are infinite values that should be replaced

In [ ]:
# np.isinf(df_merchant_id_group).any()
# np.isinf(df_merchant_id_group).any().value_counts()

# print("printing column name where infinity is present")
# col_name = df_merchant_id_group.columns.to_series()[np.isinf(df_merchant_id_group).any()]
# print(col_name)

# print(df_merchant_id_group['merchants_avg_purchases_lag3_sum'].median())
# print(df_merchant_id_group['merchants_avg_purchases_lag3_mean'].median())
# print(df_merchant_id_group['merchants_avg_purchases_lag6_sum'].median())
# print(df_merchant_id_group['merchants_avg_purchases_lag6_mean'].median())
# print(df_merchant_id_group['merchants_avg_purchases_lag12_sum'].median())
# print(df_merchant_id_group['merchants_avg_purchases_lag12_mean'].median())

# missing_value_index = df_merchant_id_group.loc[np.isinf(df_merchant_id_group['merchants_avg_purchases_lag3_sum']),
#                                                'merchants_avg_purchases_lag3_sum'].index.values

# missing_value_index

# df_merchant_id_group.loc[missing_value_index, 'merchants_avg_purchases_lag3_sum']

# Replace all infinite values to NaN which will replaced with median values collectively
# df_merchant_id_group.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
# merchants_data['merchant_id'].describe()

# newmerchant_with_merchants_data.columns

# merchants_data['merchant_category_id'].nunique(), newmerchant_data['merchant_category_id'].nunique()

# newmerchant_with_merchants_data.head(2)

# check_missing_values(newmerchant_with_merchants_data), newmerchant_with_merchants_data.shape

In [ ]:
# # Define the aggregation procedure outside of the groupby operation
# aggregations = {
#     'purchase_amount': ['sum', 'mean', 'std', 'min', 'max', 'size', 'median']
# }

# grouped = newmerchant_data.groupby('card_id').agg(aggregations)
# grouped.columns = grouped.columns.droplevel(level=0)
# grouped.rename(columns={
#     "sum": "sum_purchase_amount", 
#     "mean": "mean_purchase_amount",
#     "std": "std_purchase_amount", 
#     "min": "min_purchase_amount",
#     "max": "max_purchase_amount", 
#     "size": "num_purchase_amount",
#     "median": "median_purchase_amount"
# }, inplace=True)
# grouped.reset_index(inplace=True)

# grouped.shape

# df_train = pd.merge(df_train, grouped, on="card_id", how="left")
# df_test = pd.merge(df_test, grouped, on="card_id", how="left")

# del grouped
# gc.collect()

In [ ]:
# Due to card_ids not found in newmerchant_data
# check_missing_values(df_train), check_missing_values(df_test)

In [ ]:
# newmerchant_with_merchants_data.columns

## Merging with 'newmerchant_data' with 'card_id'

In [ ]:
newmerchant_data['month_lag'].value_counts()

In [ ]:
%%time
aggs = {}

# 'year', 'month', 'weekofyear', 'dayofweek', 'hour', 'merchant_id', 'subsector_id',
# for col in ['merchant_category_id']:
#             # 'city_id', 'state_id']: # added more features
#     aggs[col] = ['nunique']

# KS discard features 'purchase_amount' ['sum', 'mean', 'median']
aggs['purchase_amount'] = ['min', 'max'] # 'std'
aggs['installments'] = ['sum', 'max'] # ['min','mean','var'], ['std', 'median']
aggs['purchase_date'] = ['min', 'max']
aggs['month_lag'] = ['mean'] # ['max','min','mean','var'], 'std'

aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

# aggs['weekend'] = ['mean'] # 'sum',
aggs['purchase_date_weekend'] = ['mean'] # 'sum',
# aggs['month_diff'] = ['mean']
aggs['purchase_date_month_diff'] = ['mean'] # want to try 'sum'

# Check features if can achieve better performance; want to try
# Some of the aggregations do not improve the overall RMSE from 130 KS features
# aggs['purchase_date_year'] = ['nunique', 'sum', 'mean']
# aggs['purchase_date_weekofyear'] = ['nunique', 'sum'] # 'mean'
# aggs['purchase_date_month'] = ['nunique', 'sum', 'mean']
# aggs['purchase_date_dayofweek'] = ['nunique', 'sum', 'mean']
# aggs['purchase_date_hour'] = ['nunique', 'mean'] # 'sum'

# purchase amount aggregations
# KS discard features 'card_sum_category_1_0_purchase_amount_mean', 'card_sum_category_1_0_purchase_amount_sum'
# aggs['card_sum_category_1_0_purchase_amount'] = ['mean', 'sum'] # 'std'
# KS discard features 'card_sum_category_1_1_purchase_amount_mean'
aggs['card_sum_category_1_1_purchase_amount'] = ['sum'] # 'std', 'mean'
# KS discard features 'card_sum_month_lag_1_purchase_amount_sum', 'card_sum_month_lag_1_purchase_amount_mean'
# aggs['card_sum_month_lag_1_purchase_amount'] = ['mean', 'sum'] # 'std'
# KS discard features 'card_sum_month_lag_2_purchase_amount_sum', 'card_sum_month_lag_2_purchase_amount_mean'
# aggs['card_sum_month_lag_2_purchase_amount'] = ['mean', 'sum'] # 'std'

# 'card_sum_authorized_flag_1_purchase_amount_sum', 'card_sum_authorized_flag_1_purchase_amount_mean'
# aggs['card_sum_authorized_flag_1_purchase_amount'] = ['mean', 'sum'] # 'std'

# word2vec
# KS discard features
# aggs['w2v_new_city_id_0'] = ['std']
# aggs['w2v_new_city_id_1'] = ['std']
# aggs['w2v_new_city_id_2'] = ['std']
# aggs['w2v_new_city_id_3'] = ['std']
# aggs['w2v_new_city_id_4'] = ['std']

# aggs['w2v_new_subsector_id_0'] = ['std']
# aggs['w2v_new_subsector_id_1'] = ['std']
# aggs['w2v_new_subsector_id_2'] = ['std']
# aggs['w2v_new_subsector_id_3'] = ['std']
# aggs['w2v_new_subsector_id_4'] = ['std']

# aggs['w2v_new_merchant_id_0'] = ['std']
# aggs['w2v_new_merchant_id_1'] = ['std']
# aggs['w2v_new_merchant_id_2'] = ['std']
# aggs['w2v_new_merchant_id_3'] = ['std']
# aggs['w2v_new_merchant_id_4'] = ['std']

# aggs['w2v_new_merchant_category_id_0'] = ['std']
# aggs['w2v_new_merchant_category_id_1'] = ['std']
# aggs['w2v_new_merchant_category_id_2'] = ['std']
# aggs['w2v_new_merchant_category_id_3'] = ['std']
# aggs['w2v_new_merchant_category_id_4'] = ['std']

aggs['w2v_new_state_id_0'] = ['std']
aggs['w2v_new_state_id_1'] = ['std']
aggs['w2v_new_state_id_2'] = ['std']
aggs['w2v_new_state_id_3'] = ['std']
aggs['w2v_new_state_id_4'] = ['std']

## To be continue for further feature explorations
# Check features if can achieve better performance
# aggs['merchant_id'] = ['std'] # 'nunique' gives error
# aggs['merchant_category_id'] = ['std'] # 'nunique' gives error
# Explore category_1_purchase_amount_sum_mean
# aggs['category_1_purchase_amount_sum'] = ['mean', 'sum']
# aggs['category_1_purchase_amount_mean'] = ['mean', 'sum']

# aggs['merchants_most_recent_sales_range_min'] = ['min', 'max']
aggs['merchants_most_recent_sales_range_max'] = ['min']  # 'max'
aggs['merchants_most_recent_purchases_range_min'] = ['min', 'max']
aggs['merchants_most_recent_purchases_range_max'] = ['min'] # 'max'

aggs['merchants_category_2_mean'] = ['sum'] # , 'min', 'max'

# aggs['merchants_merchant_category_id_sum'] = ['sum', 'mean', 'min', 'max'] # 'size'
# aggs['merchants_merchant_category_id_mean'] = ['sum', 'mean', 'min', 'max'] # 'size'
# aggs['merchants_merchant_category_id_size'] = ['sum', 'mean'] # 'size', 'min', 'max'
# aggs['merchants_merchant_category_id_min'] = ['sum', 'mean', 'min', 'max'] # 'size'
# aggs['merchants_merchant_category_id_max'] = ['sum', 'mean'] # 'size', 'min', 'max'

# Explore merchants_category_1
# aggs['merchants_category_1_min'] = ['min', 'max', 'sum']
# aggs['merchants_category_1_max'] = ['sum'] # 'min', 'max'
# aggs['merchants_category_1_nunique'] = ['min', 'max']
##

# include columns with merchant information
# aggs['merchants_merchant_group_id_nunique'] = ['size']
# aggs['merchants_subsector_id_nunique'] = ['size']
# aggs['merchants_category_1_nunique'] = ['size']
# aggs['merchants_most_recent_sales_range_nunique'] = ['size']
# aggs['merchants_most_recent_purchases_range_nunique'] = ['size']
# aggs['merchants_category_4_nunique'] = ['size']
# aggs['merchants_city_id_nunique'] = ['size']
# aggs['merchants_state_id_nunique'] = ['size']

# merchant_category_id
# Low feature importance; might need to remove
# aggs['merchants_merchant_category_id_std'] = ['min', 'max', 'std']

# numerical_1
# KS discard features 'numerical_1_mean_max'
aggs['merchants_numerical_1_mean'] = ['min']
# KS discard features 'numerical_1_sum_max', 'numerical_1_sum_mean'
aggs['merchants_numerical_1_sum'] = ['min']

# numerical_2
# KS discard features 'numerical_2_mean_max'
aggs['merchants_numerical_2_mean'] = ['min']
# KS discard features 'numerical_2_sum_mean'
# aggs['merchants_numerical_2_sum'] = ['mean'] # 'min', 'max', 

# 'avg_sales_lag3'
# KS discard features 'avg_sales_lag3_sum_mean'
aggs['merchants_avg_sales_lag3_sum'] = ['min', 'max'] # 'mean'
aggs['merchants_avg_sales_lag3_mean'] = ['min', 'max']
aggs['merchants_avg_sales_lag3_min'] = ['min', 'max']
# aggs['merchants_avg_sales_lag3_size'] = ['min', 'max']
# aggs['merchants_avg_sales_lag3_max'] = ['min', 'max']

# 'avg_purchases_lag3'
# KS discard features 'avg_purchases_lag3_sum_max', 'avg_purchases_lag3_sum_mean'
aggs['merchants_avg_purchases_lag3_sum'] = ['min'] # 'max', 'mean'
# KS discard features 'avg_purchases_lag3_mean_max'
aggs['merchants_avg_purchases_lag3_mean'] = ['min'] # 'max'
# KS discard features 'avg_purchases_lag3_min_min', 'avg_purchases_lag3_min_max'
# aggs['merchants_avg_purchases_lag3_min'] = ['min', 'max']
# KS discard features'avg_purchases_lag3_max_max'
aggs['merchants_avg_purchases_lag3_max'] = ['min'] # 'max'
# aggs['merchants_avg_purchases_lag3_size'] = ['min', 'max']

## 'active_months_lag3'
# aggs['merchants_active_months_lag3_sum'] = ['mean'] # 'min', 'max',
# aggs['merchants_active_months_lag3_mean'] = ['mean', 'median'] # 'sum', 'min', 'max'
# aggs['merchants_active_months_lag3_size'] = ['min', 'max']
# aggs['merchants_active_months_lag3_min'] = ['min', 'max']
# aggs['merchants_active_months_lag3_max'] = ['min', 'max']

# 'avg_sales_lag6'
# KS discard features 'avg_sales_lag6_sum_mean'
aggs['merchants_avg_sales_lag6_sum'] = ['min', 'max'] # 'mean'
aggs['merchants_avg_sales_lag6_mean'] = ['min', 'max']
aggs['merchants_avg_sales_lag6_min'] = ['min', 'max']
# aggs['merchants_avg_sales_lag6_size'] = ['min', 'max']
# aggs['merchants_avg_sales_lag6_max'] = ['min', 'max']

# 'avg_purchases_lag6'
# KS discard features 'avg_purchases_lag6_sum_min', 'avg_purchases_lag6_sum_max', 'avg_purchases_lag6_sum_mean'
# aggs['merchants_avg_purchases_lag6_sum'] = ['min', 'max', 'mean']
# KS discard features 'avg_purchases_lag6_mean_min', 'avg_purchases_lag6_max_max'
# aggs['merchants_avg_purchases_lag6_mean'] = ['min'] # 'max'
# aggs['merchants_avg_purchases_lag6_max'] = ['max'] # 'min'
aggs['merchants_avg_purchases_lag6_min'] = ['min'] # 'max'
# aggs['merchants_avg_purchases_lag6_size'] = ['min', 'max']

# 'active_months_lag6'
# aggs['merchants_active_months_lag6_sum'] = ['mean', 'min', 'max']
# aggs['merchants_active_months_lag6_mean'] = ['sum', 'min', 'max']
# aggs['merchants_active_months_lag6_size'] = ['min', 'max']
# aggs['merchants_active_months_lag6_min'] = ['min', 'max']
# aggs['merchants_active_months_lag6_max'] = ['min', 'max']

# 'avg_sales_lag12'
# KS discard features 'avg_sales_lag12_sum_sum', 'avg_sales_lag12_sum_max',
aggs['merchants_avg_sales_lag12_sum'] = ['min', 'mean'] # 'sum', 'max'
aggs['merchants_avg_sales_lag12_mean'] = ['min', 'max']
# aggs['merchants_avg_sales_lag12_size'] = ['min', 'max']
# aggs['merchants_avg_sales_lag12_min'] = ['min', 'max']
# aggs['merchants_avg_sales_lag12_max'] = ['min', 'max']

# 'avg_purchases_lag12'
# KS discard features 'avg_purchases_lag12_sum_max', 'avg_purchases_lag12_sum_mean'
# aggs['merchants_avg_purchases_lag12_sum'] = ['max', 'mean'] # 'min'
# 'avg_purchases_lag12_mean_min', 'avg_purchases_lag12_mean_max'
# aggs['merchants_avg_purchases_lag12_mean'] = ['min', 'max']
# KS discard features 'avg_purchases_lag12_min_min', 'avg_purchases_lag12_min_max'
# aggs['merchants_avg_purchases_lag12_min'] = ['min', 'max']
# aggs['merchants_avg_purchases_lag12_size'] = ['min', 'max'] 
# aggs['merchants_avg_purchases_lag12_max'] = ['min', 'max']


# 'active_months_lag12'
aggs['merchants_active_months_lag12_mean'] = ['sum'] # 'min', 'max'
# aggs['merchants_active_months_lag12_sum'] = ['mean', 'min', 'max']
# aggs['merchants_active_months_lag12_size'] = ['min', 'max']
# aggs['merchants_active_months_lag12_min'] = ['min', 'max']
# aggs['merchants_active_months_lag12_max'] = ['min', 'max']

# added more features with median for 'installments'
# aggs['month'] = [ 'min', 'max', 'mean', 'var']

# 'category_1_mean_mean' not useful
for col in ['category_2','category_3']: # 'category_1'
    # Replace newmerchant_data with newmerchant_with_merchants_data
    newmerchant_with_merchants_data[col+'_purchase_amount_mean'] = newmerchant_with_merchants_data.groupby([col])['purchase_amount'].transform('mean')
    aggs[col+'_purchase_amount_mean'] = ['mean']

# Explore purchase_amount_sum_mean in 'category_1' which helps to improve on ranking
for col in ['category_1']:
    newmerchant_with_merchants_data[col+'_purchase_amount_sum'] = newmerchant_with_merchants_data.groupby([col])['purchase_amount'].transform('sum')
    aggs[col+'_purchase_amount_sum'] = ['mean'] 

new_columns = create_new_columns('new_merchant',aggs)
print("New aggregated columns :\n", new_columns)
df_new_merchant_trans_group = newmerchant_with_merchants_data.groupby('card_id').agg(aggs) # newmerchant_data

df_new_merchant_trans_group.columns = new_columns
df_new_merchant_trans_group.reset_index(drop=False,inplace=True)
df_new_merchant_trans_group['new_merchant_purchase_date_diff'] = (df_new_merchant_trans_group['new_merchant_purchase_date_max'] - df_new_merchant_trans_group['new_merchant_purchase_date_min']).dt.days
df_new_merchant_trans_group['new_merchant_purchase_date_average'] = df_new_merchant_trans_group['new_merchant_purchase_date_diff'] / df_new_merchant_trans_group['new_merchant_card_id_size']

# Replace datetime.datetime.today() with df_new_merchant_trans_group['new_merchant_purchase_date_max'] in attempt to improve relevancy
df_new_merchant_trans_group['new_merchant_purchase_date_uptonow'] = (datetime.datetime.today() - \
                                                                     df_new_merchant_trans_group['new_merchant_purchase_date_max']).dt.days

# Explore new features for rank ascend
df_new_merchant_trans_group['new_card_last_woy'] = df_new_merchant_trans_group['new_merchant_purchase_date_max'].dt.weekofyear
df_new_merchant_trans_group['new_card_last_doy'] = df_new_merchant_trans_group['new_merchant_purchase_date_max'].dt.dayofyear
df_new_merchant_trans_group['new_card_last_day'] = df_new_merchant_trans_group['new_merchant_purchase_date_max'].dt.day

# --
# Reference : https://www.kaggle.com/raddar/towards-de-anonymizing-the-data-some-insights
df_new_merchant_trans_group['new_merchant_purchase_amount_new'] = np.round((newmerchant_data['purchase_amount'] / 0.00150265118) + 497.06, 8)

# Check new feature effects on RMSE
target = 'new_merchant_purchase_amount_new'
df_new_merchant_trans_group['new_merchant_card_sum_purchase_amount_new'] = df_new_merchant_trans_group['card_id'].map(df_new_merchant_trans_group.groupby(['card_id'])[target].sum())

df_new_merchant_trans_group['new_merchant_card_mean_purchase_amount_new'] = df_new_merchant_trans_group['card_id'].map(df_new_merchant_trans_group.groupby(['card_id'])[target].mean())
df_new_merchant_trans_group['new_merchant_card_median_purchase_amount_new'] = df_new_merchant_trans_group['card_id'].map(df_new_merchant_trans_group.groupby(['card_id'])[target].median())
df_new_merchant_trans_group['new_merchant_card_min_purchase_amount_new'] = df_new_merchant_trans_group['card_id'].map(df_new_merchant_trans_group.groupby(['card_id'])[target].min())
df_new_merchant_trans_group['new_merchant_card_max_purchase_amount_new'] = df_new_merchant_trans_group['card_id'].map(df_new_merchant_trans_group.groupby(['card_id'])[target].max())
df_new_merchant_trans_group['new_merchant_card_std_purchase_amount_new'] = df_new_merchant_trans_group['card_id'].map(df_new_merchant_trans_group.groupby(['card_id'])[target].std())

# KS discard feature 
# 'new_merchant_card_sum_month_lag_1_purchase_amount_new', 'new_merchant_card_sum_month_lag_2_purchase_amount_new'
# df_new_merchant_trans_group['month_lag'] = newmerchant_data['month_lag']
# df_new_merchant_trans_group['new_merchant_card_sum_month_lag_1_purchase_amount_new'] =\
#     df_new_merchant_trans_group['card_id'].map(df_new_merchant_trans_group[df_new_merchant_trans_group['month_lag']==1].groupby(['card_id'])[target].sum())
# df_new_merchant_trans_group['new_merchant_card_sum_month_lag_2_purchase_amount_new'] =\
#     df_new_merchant_trans_group['card_id'].map(df_new_merchant_trans_group[df_new_merchant_trans_group['month_lag']==2].groupby(['card_id'])[target].sum())

# Explore new building feature, 'card_purchase_date_max_ratio', for rank ascend
df_new_merchant_trans_group['new_purchase_date_int'] = newmerchant_with_merchants_data['purchase_date'].astype(np.int64) * 1e-9
df_new_merchant_trans_group['new_card_purchase_date_max'] = \
        df_new_merchant_trans_group.groupby(['card_id'])['new_purchase_date_int'].transform('max')
# --

print()
print("Shape of df_new_merchant_trans_group :\n", df_new_merchant_trans_group.shape)
print("df_new_merchant_trans_group columns :\n", df_new_merchant_trans_group.columns)

# Remove aggs['merchants_merchant_category_id_std'] = ['min', 'max', 'std'] in v54

## Create more aggregated features from 'new_merchant_purchase_amount_new

In [ ]:
# df_new_merchant_trans_group['new_merchant_purchase_date_diff']
check_missing_values(df_new_merchant_trans_group)

In [ ]:
df_train.shape, df_test.shape

In [ ]:
# Get ready for training models
X_train = df_train.merge(df_new_merchant_trans_group,on='card_id',how='left')
X_test = df_test.merge(df_new_merchant_trans_group,on='card_id',how='left')

X_train.shape, X_test.shape

In [ ]:
# Release unnecessary data to reduce memory used
del df_new_merchant_trans_group, merchants_data, df_train, df_test
gc.collect()

In [ ]:
# X_train['hist_purchase_date_max'].head(2), X_test['hist_purchase_date_max'].head(2)

# X_train['new_merchant_purchase_date_max'].head(2), X_test['new_merchant_purchase_date_max'].head(2)

# X_train['hist_purchase_amount_max'].head(2), X_test['hist_purchase_amount_max'].head(2)

# X_train['new_merchant_purchase_amount_max'].head(2), X_test['new_merchant_purchase_amount_max'].head(2)

In [ ]:
# df_train = df_train.merge(df_merchant_category, on='merchant_category_id', how='left')
# df_test = df_test.merge(df_merchant_category, on='merchant_category_id', how='left')

# df_train.shape, df_test.shape
# check_missing_values(X_train), check_missing_values(X_test)

In [ ]:
# Concatenate the categorical features
# Numerical representations of the nominal feature_1, feature_2, feature_3
# X_train = pd.concat([X_train, ohe_train_df_1, ohe_train_df_2, ohe_train_df_3], axis=1, sort=False)
# X_test = pd.concat([X_test, ohe_test_df_1, ohe_test_df_2, ohe_test_df_3], axis=1, sort=False)

# del ohe_train_df_1, ohe_train_df_2, ohe_train_df_3
# del ohe_test_df_1, ohe_test_df_2, ohe_test_df_3
# gc.collect()

In [ ]:
# X_train[['hist_purchase_date_min', 'hist_purchase_date_max', 'new_merchant_purchase_date_min', 'new_merchant_purchase_date_max']]
# X_train[['hist_purchase_date_min', 'hist_purchase_date_max', 'new_merchant_purchase_date_min', 'new_merchant_purchase_date_max']].isna().any()
# X_test[['hist_purchase_date_min', 'hist_purchase_date_max', 'new_merchant_purchase_date_min', 'new_merchant_purchase_date_max']].isna().any()

# print("Missing purchase date (min-max) in X_train : ")
# print("min mode : {}\nmax mode : {}".format(X_train['new_merchant_purchase_date_min'].mode(),
#                                             X_train['new_merchant_purchase_date_max'].mode()))
# print("-" * 80)
# print("Missing purchase date (min-max) in X_test : ")
# print("min mode : {}\nmax mode : {}".format(X_test['new_merchant_purchase_date_min'].mode(),
#                                             X_test['new_merchant_purchase_date_max'].mode()))

# type(X_train['new_merchant_purchase_date_min'].mode()[0]), type(X_train['new_merchant_purchase_date_max'].mode()[0])

# print("Purchase data in historical_data")
# print(historical_data['purchase_date'].min(), historical_data['purchase_date'].max())
# print("\nPurchase data in newmerchant_data")
# print(newmerchant_data['purchase_date'].min(), newmerchant_data['purchase_date'].max())

#### Impute missing purchase min-max values in train and test data

In [ ]:
# X_train[['hist_purchase_date_min', 'hist_purchase_date_max', 'new_merchant_purchase_date_min', 'new_merchant_purchase_date_max',
#          'hist_purchase_amount_min', 'hist_purchase_amount_max', 'new_merchant_purchase_amount_min', 'new_merchant_purchase_amount_max']].info()

# X_test[['hist_purchase_date_min', 'hist_purchase_date_max', 'new_merchant_purchase_date_min', 'new_merchant_purchase_date_max',
#         'hist_purchase_amount_min', 'hist_purchase_amount_max', 'new_merchant_purchase_amount_min', 'new_merchant_purchase_amount_max']].info()

In [ ]:
X_train['hist_purchase_date_min'].fillna(historical_data['purchase_date'].min(), inplace = True)
X_train['hist_purchase_date_max'].fillna(historical_data['purchase_date'].max(), inplace = True)
X_train['new_merchant_purchase_date_min'].fillna(newmerchant_data['purchase_date'].min(), inplace = True)
X_train['new_merchant_purchase_date_max'].fillna(newmerchant_data['purchase_date'].max(), inplace = True)

X_train['hist_purchase_amount_min'].fillna(historical_data['purchase_amount'].min(), inplace = True)
X_train['hist_purchase_amount_max'].fillna(historical_data['purchase_amount'].max(), inplace = True)
X_train['new_merchant_purchase_amount_min'].fillna(newmerchant_data['purchase_amount'].min(), inplace = True)
X_train['new_merchant_purchase_amount_max'].fillna(newmerchant_data['purchase_amount'].max(), inplace = True)

X_test['hist_purchase_date_min'].fillna(historical_data['purchase_date'].min(), inplace = True)
X_test['hist_purchase_date_max'].fillna(historical_data['purchase_date'].max(), inplace = True)
X_test['new_merchant_purchase_date_min'].fillna(newmerchant_data['purchase_date'].min(), inplace = True)
X_test['new_merchant_purchase_date_max'].fillna(newmerchant_data['purchase_date'].max(), inplace = True)

X_test['hist_purchase_amount_min'].fillna(historical_data['purchase_amount'].min(), inplace = True)
X_test['hist_purchase_amount_max'].fillna(historical_data['purchase_amount'].max(), inplace = True)
X_test['new_merchant_purchase_amount_min'].fillna(newmerchant_data['purchase_amount'].min(), inplace = True)
X_test['new_merchant_purchase_amount_max'].fillna(newmerchant_data['purchase_amount'].max(), inplace = True)

In [ ]:
# # 'hist_card_sum_month_lag_0_purchase_amount_new'
# target = 'hist_sum_purchase_amount_new'
# X_train['month_lag'] = historical_data['month_lag']
# X_train['hist_card_sum_month_lag_0_purchase_amount_new'] =\
#     X_train['card_id'].map(X_train[X_train['month_lag']==0].groupby(['card_id'])[target].sum())
# TBD

In [ ]:
# Releases all memories in preparation for model training
del historical_data, newmerchant_data # df_new_merchant_trans_group, merchants_data, df_train, df_test, train_data, test_data
gc.collect()

In [ ]:
X_train[['hist_purchase_date_min', 'hist_purchase_date_max', 'new_merchant_purchase_date_min', 'new_merchant_purchase_date_max',
         'hist_purchase_amount_min', 'hist_purchase_amount_max', 'new_merchant_purchase_amount_min', 'new_merchant_purchase_amount_max']].info()

In [ ]:
X_test[['hist_purchase_date_min', 'hist_purchase_date_max', 'new_merchant_purchase_date_min', 'new_merchant_purchase_date_max',
        'hist_purchase_amount_min', 'hist_purchase_amount_max', 'new_merchant_purchase_amount_min', 'new_merchant_purchase_amount_max']].info()

In [ ]:
# The ratios does not improve the overall RMSE
# purchase_date in timestamp format
# for df in [X_train, X_test]:
#     df['hist_purchase_date_min_month'] = df['hist_purchase_date_min'].dt.month
#     df['hist_purchase_date_max_month'] = df['hist_purchase_date_max'].dt.month
#     df['new_merchant_purchase_date_min_month'] = df['new_merchant_purchase_date_min'].dt.month
#     df['new_merchant_purchase_date_max_month'] = df['new_merchant_purchase_date_max'].dt.month
    
#     df['hist_purchase_date_maxmin_month_ratio'] = df['hist_purchase_date_max_month'] / df['hist_purchase_date_min_month']
#     df['new_merchant_purchase_date_maxmin_month_ratio'] = df['new_merchant_purchase_date_max_month'] / \
#                                                           df['new_merchant_purchase_date_min_month']

# X_train.shape, X_test.shape

## More Feature Engineering with 'first_active_month'

In [ ]:
for df in [X_train, X_test]:
    df_first_active_month_max = pd.to_datetime(df['first_active_month'].max())
    
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['first_active_month_dayofweek'] = df['first_active_month'].dt.dayofweek
    df['first_active_month_weekofyear'] = df['first_active_month'].dt.weekofyear
    df['first_active_month_month'] = df['first_active_month'].dt.month
    
    # df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
    # KS discard feature 'elapsed_time' = ((df_first_active_month_max - df['first_active_month']).dt.days)//30
    df['first_active_month_today_elapsed_time'] = ((datetime.datetime.today() - df['first_active_month']).dt.days)//30
    
    # Explore interactions with 'hist_purchase_date_min', 'new_merchant_purchase_date_min' and 'first_active_month'
    df['hist_pdmin_active_month_diff'] = ((df['hist_purchase_date_min'] - df['first_active_month']).dt.days)//30
    df['new_merchant_pdmin_active_month_diff'] = ((df['new_merchant_purchase_date_min'] - df['first_active_month']).dt.days)//30

X_train.shape, X_test.shape

In [ ]:
# Convert Timestamp to int64 with imputed missing values
for f in ['hist_purchase_date_min', 'hist_purchase_date_max', 'new_merchant_purchase_date_min', 'new_merchant_purchase_date_max']:
    print(type(X_train[f][0]))
    print(type(X_test[f][0]))
    X_train[f] = X_train[f].astype(np.int64) * 1e-9
    X_test[f] = X_test[f].astype(np.int64) * 1e-9

In [ ]:
# X_train[['hist_card_id_size', 'new_merchant_card_id_size']].head(2)
# X_train[['new_merchant_card_id_size', 'new_merchant_purchase_date_max']].head(2)

* # Feature Interactions (ratios and differences)

In [ ]:
# List all columns in dataframe
# list(X_train.columns.values)

In [ ]:
# This strong 'new_hist_purchase_date_max_ratio' feature elevates the ranking by 10%+
# Added 10 new KS features
for df in [X_train, X_test]:
    # purchase_date
    df['new_hist_purchase_date_max_ratio'] = df['new_merchant_purchase_date_max'] / df['hist_purchase_date_max']
    df['new_hist_purchase_date_min_ratio'] = df['new_merchant_purchase_date_min'] / df['hist_purchase_date_min']
    df['hist_purchase_date_maxmin_ratio'] = df['hist_purchase_date_max'] / df['hist_purchase_date_min']
    df['new_purchase_date_maxmin_ratio'] = df['new_merchant_purchase_date_max'] / df['new_merchant_purchase_date_min']
    
    # purchase_amount
    df['new_hist_purchase_amount_max_diff'] = df['new_merchant_purchase_amount_max'] - df['hist_purchase_amount_max']
    df['new_hist_purchase_amount_min_diff'] = df['new_merchant_purchase_amount_min'] - df['hist_purchase_amount_min']
    df['new_hist_purchase_amount_max_ratio'] = df['new_merchant_purchase_amount_max'] / df['hist_purchase_amount_max']
    df['new_hist_purchase_amount_min_ratio'] = df['new_merchant_purchase_amount_min'] / df['hist_purchase_amount_min']
    
    df['new_pa_max_hist_pa_sum_ratio'] = df['new_merchant_purchase_amount_max'] / df['hist_purchase_amount_sum']
    df['new_hist_card_id_total'] = df['new_merchant_card_id_size'] + df['hist_card_id_size']

    # Need to produce 'card_purchase_date_max_ratio'

X_train.shape, X_test.shape

In [ ]:
# Did not improve on the RMSE ranking
for df in [X_train, X_test]:
    df['new_hist_purchase_date_uptonow_ratio'] = df['new_merchant_purchase_date_uptonow'] / df['hist_purchase_date_uptonow']
    df['new_hist_purchase_date_uptonow_diff'] = df['new_merchant_purchase_date_uptonow'] - df['hist_purchase_date_uptonow']
    
X_train.shape, X_test.shape

In [ ]:
# X_train[['new_card_purchase_date_max', 'hist_card_purchase_date_max']].head(2)

In [ ]:
# Did not improve on the RMSE ranking
X_train['card_purchase_date_max_ratio'] = X_train['new_card_purchase_date_max'] + X_train['hist_card_purchase_date_max']
X_test['card_purchase_date_max_ratio'] = X_test['new_card_purchase_date_max'] + X_test['hist_card_purchase_date_max']

X_train.shape, X_test.shape

In [ ]:
# %%time
# features_correlation = pd.DataFrame(historical_data, columns=['category_1','city_id',
#                                                               'category_2','state_id',
#                                                               'category_3','installments'])

# colormap = plt.cm.RdBu
# plt.figure(figsize=(10,8))
# plt.title('Pearson Correlation of anonymise features', y=1.05, size=15)
# sns.heatmap(features_correlation.astype(float).corr(),linewidths=0.1,vmax=1.0, 
#             square=True, cmap=colormap, linecolor='white', annot=True)

In [ ]:
# Did not improve on the RMSE ranking
# for f in ['new_merchant_purchase_date_max',
#           'hist_purchase_date_max',]:
#     X_train[f +'_int'] = X_train[f].astype(np.int64) * 1e-9
#     X_test[f +'_int'] = X_test[f].astype(np.int64) * 1e-9
    
# X_train['card_purchase_date_max_ratio'] = X_train['new_merchant_purchase_date_max_int'] / X_train['hist_purchase_date_max_int']
# X_test['card_purchase_date_max_ratio'] = X_test['new_merchant_purchase_date_max_int'] / X_test['hist_purchase_date_max_int']

In [ ]:
# X_train['new_merchant_merchants_category_1_min_min'].mode()[0]

# # Fill missing values for min-max purchase date newmerchant_data from after merging with newmmerchant_data for the card_ids in train_data
# X_train['new_merchant_merchants_category_1_min_min'].fillna(newmerchant_data['category_1'].min(), inplace = True)
# X_train['new_merchant_merchants_category_1_min_max'].fillna(newmerchant_data['category_1'].min(), inplace = True)
# X_train['new_merchant_merchants_category_1_max_min'].fillna(newmerchant_data['category_1'].max(), inplace = True)
# X_train['new_merchant_merchants_category_1_max_max'].fillna(newmerchant_data['category_1'].max(), inplace = True)

# X_test['new_merchant_merchants_category_1_min_min'].fillna(newmerchant_data['category_1'].min(), inplace = True)
# X_test['new_merchant_merchants_category_1_min_max'].fillna(newmerchant_data['category_1'].min(), inplace = True)
# X_test['new_merchant_merchants_category_1_max_min'].fillna(newmerchant_data['category_1'].max(), inplace = True)
# X_test['new_merchant_merchants_category_1_max_max'].fillna(newmerchant_data['category_1'].max(), inplace = True)

# X_train[['new_merchant_merchants_category_1_min_min', 'new_merchant_merchants_category_1_min_max',
#          'new_merchant_merchants_category_1_max_min', 'new_merchant_merchants_category_1_max_max']].info()

In [ ]:
# excluded_features = ['first_active_month', 'card_id', 'target', 'date', 'year']
# Excluse non numeric features
# excluded_features = ['first_active_month', 'card_id', 'target', 'hist_purchase_date_min', 'hist_purchase_date_max']
# train_features = [c for c in df_train.columns if c not in excluded_features]

# 'hist_purchase_date_min', 'hist_purchase_date_max', 'new_merchant_purchase_date_min', 'new_merchant_purchase_date_max'
# Consider removal of these 'new_merchant_purchase_date_max', 'hist_purchase_date_max' after getting their ratio
excluded_features = ['first_active_month', 'card_id', 'target', 'outliers']
train_features = [c for c in X_train.columns if c not in excluded_features]

print("{} features used for training :\n{}".format(len(train_features), train_features))

In [ ]:
# Filled mean information for the missing cards from newmerchant_data
# Fill missing values with mean values; maybe use median value
for col in train_features:
    for df in [X_train, X_test]:
        if df[col].dtype == "float64":
            df[col] = df[col].fillna(df[col].median()) # mean

In [ ]:
X_train.shape, X_test.shape

In [ ]:
check_missing_values(X_train), check_missing_values(X_test)

In [ ]:
# Need to remove 'hist_card_std_purchase_amount_new', 'new_merchant_card_std_purchase_amount_new'
X_train.drop(columns=['hist_card_std_purchase_amount_new', 'new_merchant_card_std_purchase_amount_new'], axis=1, inplace=True)
X_test.drop(columns=['hist_card_std_purchase_amount_new', 'new_merchant_card_std_purchase_amount_new'], axis=1, inplace=True)
X_train.shape, X_test.shape

* # Kolmogorov-Smirnov
> - It computes on 2 samples to test for the null hypothesis that 2 independent samples are drawn from the same continuous distribution.</br>
> - The statistic test computes the KS statistic and pvalue. If the p value is less than .1 the null hypothesis is rejected.</br>
> - Reducing from 190 to 126 features gives a better scoring (from 3.69219 to 3.69101) with p value is less than 0.1.</br>

In [ ]:
%%time
from tqdm import tqdm
# Kolmogorov-Smirnov test for goodness of fit
# In this case we are sampling values from corresponding columns in train and test data frame followed by storing their pvalues in Se.
# If the p value is less than 0.05 (0.1) the null hypothesis is rejected.
# Columns with pvalues less than 0.05 (0.1) are added to the list_discarded.
list_p_value =[]

for i in tqdm(X_train[train_features].columns):
    list_p_value.append(ks_2samp(X_train[i] , X_test[i])[1])

Se = pd.Series(list_p_value, index = X_train[train_features].columns).sort_values() 
list_discarded = list(Se[Se < 0.1].index)

# 'hist_category_3_purchase_amount_sum_mean' and 'hist_purchase_amount_max'
list_discarded

In [ ]:
X_train.shape, X_test.shape

In [ ]:
len(list_discarded)

- #### Code snippets and data observations 

In [ ]:
# There are card_id in train_data but not in newmerchant_data
# newmerchant_data.loc[newmerchant_data['card_id'] == missing_sum_card_id]

In [ ]:
# There are duplicate card_ids in historical_data with 29,112,361 transactions, unique values are 325,540
# There are missing values in category_3', 'merchant_id' and 'category_2'
# Replace missing values with mode values for category_3', 'merchant_id' and 'category_2'
# When inplace = True, the data is modified in place,
# which means it will return nothing and the dataframe is now updated.

In [ ]:
# Need to impute values which is the challenge for prediction

# Stable documentation : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html#pandas.merge
# Merge train_data[['card_id','target']] with historical_data to give new historical_data
# merging target value of card_id for each transaction in historical_transactions Data; using union of keys from both frames
# historical_data = pd.merge(historical_data, train_data[['card_id','target']], how = 'outer', on = 'card_id')

# # merging target value of card_id for each transaction in new_merchants_transactions Data
# newmerchant_data = pd.merge(newmerchant_data, train_data[['card_id','target']], how = 'outer', on = 'card_id')

# historical_data['target'].isnull().sum() # / historical_data.shape[0] = 38.07%
# historical_data['target'].describe()

# for row in range(historical_data.shape[0]):
#   if (historical_data['card_id'][row] == test_data['card_id'][10]):
#     print(row)

# for card_id[0], 'C_ID_0ab67a22ab' : there are 1304310 - 1304243 = 67 historical transactions
# for card_id[10], 'C_ID_4859ac9ed5' : there are 23622204 - 23622180 = 24 historical transactions
# historical_data.loc[[1304243]]

# reset_index() to keep the 'card_id' column that appears with X.columns
# X = partial_historical_data_target.groupby('card_id').mean().reset_index()

# groupby_card_id.isnull().sum()

# Merge test_data[['card_id']] with partial_historical_data without target to give new test_data_intersect
# test_data_intersect = pd.merge(historical_data, test_data[['card_id']], how = 'inner', on = 'card_id')
# test_data_intersect

# Drop the non-numeric features
# partial_historical_data = historical_data_target.drop(columns=['target', 'authorized_flag', 'category_1', 'category_3', 'merchant_id', 'purchase_date'])

# historical_data['target'] = historical_data['target'].replace(0, historical_data['target'].mode())

In [ ]:
check_missing_values(X_train), check_missing_values(X_test)

# Part 2 : Attempt novelty : Exploring outliers, Blending and Ensemble Learning (Stacking)

- #### Load best features, predictions and saved models for stacking and blending

In [ ]:
# Load all libraries
# Load best features from lgbm and run in GPU for DL
X_train = pd.read_csv('../input/test-predictions/X_train_137_KSfeats.csv')
X_test = pd.read_csv('../input/test-predictions/X_test_136_KSfeats.csv')
lgbm_pred_df = pd.read_csv('../input/test-predictions/lgbm_opt2_130KSfeats_K5.csv')

X_train.shape, X_test.shape, lgbm_pred_df.shape

In [ ]:
excluded_features = ['first_active_month', 'card_id', 'target', 'outliers']
train_features = [c for c in X_train.columns if c not in excluded_features]

print("{} Features used for training : \n{}".format(len(train_features), train_features))

In [ ]:
# Remove features from loaded best 137KS features in attempt to get better CV RMSE scores
list_discarded = ['hist_purchase_amount_max', 'hist_w2v_hist_merchant_id_1_std',
                  'hist_category_3_purchase_amount_sum_mean', 'hist_installments_mean']

# ['hist_purchase_amount_max', 'hist_category_3_purchase_amount_sum_mean', 'hist_installments_mean']
print("No. of train_features :", len(train_features))

features_to_remove = list_discarded
# features_to_remove = ['new_merchant_purchase_amount_sum', 'new_merchant_merchants_most_recent_purchases_range_min_max',
#                       'new_merchant_merchants_avg_purchases_lag3_sum_min', 'new_merchant_merchants_avg_sales_lag6_mean_max',
#                       'new_merchant_merchants_most_recent_sales_range_max_min', 'new_merchant_w2v_new_state_id_0_std',
#                       'new_merchant_card_id_size', 'new_merchant_w2v_new_state_id_1_std',
#                       'new_merchant_merchants_avg_sales_lag3_mean_max', 'new_merchant_merchants_avg_purchases_lag6_sum_max']

train_features = list(set(train_features) - set(features_to_remove))

print("No. of train_features after removing some low important features :", len(train_features))

len(set(features_to_remove)), len(set(train_features))

In [ ]:
# Prepare data for training and prediction
X = X_train.copy()
y = X['target'] # pandas.core.series.Series
card_ids = X_test["card_id"].copy()

X.shape, X_train.shape, X[train_features].shape, X_test[train_features].shape, y.shape

## Exploring with outliers in training data
- #### Observations :
> - The target value is almost normally distributed with bunch of outlier value near -30.
> - This distribution indicates that the target value is normalized with pre-decided mean and standard deviation.
> - 2,207 rows with target values less than -33 (1.09%) in "train.csv"
> - Values range from '-33.2' to '17.9'
> - '-33.219281' seems like an outlier point
> - Other values less than '-10' also seem like outliers due to very less in number
> - All values above '10' might also be outliers

In [ ]:
loyality_score = X['target']
ax = loyality_score.plot.hist(bins=20, figsize=(6, 5))
_ = ax.set_title("target histogram")
plt.show()

fig, axs = plt.subplots(1,2, figsize=(12, 5))
_ = loyality_score[loyality_score > 10].plot.hist(ax=axs[0])
_ = axs[0].set_title("target histogram for values greater than 10")
_ = loyality_score[loyality_score < -10].plot.hist(ax=axs[1])
_ = axs[1].set_title("target histogram for values less than -10")
plt.show()

# (A) Method 1 : LGBM with non-outliers
> - training data instances with ‘target’ < -33  are marked as outliers (2,207 of them)
> - Predictions made with test set
> - Perform the average between predictions from LGBM with non-outliers and best single model, LGBM, with complete training data</br>
(Average blending 'target' predictions from LGBM trained with non-outliers and complete data set separately)

In [ ]:
# There are 2207 rows with target values less than -30 (1.09%)
X.loc[X['target'] < -33, 'target'].value_counts()

In [ ]:
X['outliers'] = 0

# Consider the target values less than -30 are outliers
X.loc[X['target'] < -33, 'outliers'] = 1
X['outliers'].value_counts()

In [ ]:
# Getting the rows that does not have null values for 'target'
outlier_train_df = X[X['target'].notnull()]

# All rows in the training set
outlier_train_df.shape

In [ ]:
# Added a null column 'target' to X_test
X_test['target'] = np.nan
X_test.shape

In [ ]:
# Getting the rows that has null values for 'target'
outlier_test_df = X_test[X_test['target'].isnull()]

print(outlier_train_df.shape, outlier_test_df.shape, X.shape, X_test.shape)

# All rows in the test set
outlier_test_df

In [ ]:
# Getting only the 199,710 rows (non-assigned outlier with 'outliers' column == 0)
X_nonoutliers = X.loc[X['outliers'] == 0]
y_nonoutliers = X.loc[X['outliers'] == 0, 'target']

X_nonoutliers.shape, X_nonoutliers[train_features].shape, y_nonoutliers.shape

## Train model with only non-outliers to be used for average blending
- ### Build lgbm regressor with non-outliers training set
> #### Observations :
> - #### The LGBM model overfits if we just omit the rows from the training set with 'target' value less than -33.

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

np.random.seed(4590)

# Baselined lgbm params
lgbm_params = {'num_leaves': 111,
               'min_data_in_leaf': 150, # 149
               'objective':'regression',
               'max_depth': 9,
               'learning_rate': 0.005,
               "boosting": "gbdt",
               "feature_fraction": 0.7522,
               "bagging_freq": 1,
               "bagging_fraction": 0.7083,
               "bagging_seed": 11,
               "metric": 'rmse',
               "lambda_l1": 0.2634,
               "random_state": 133,
               "verbosity": -1}

# Does not improve rank with 'learning_rate': 0.001 or # 0.002
lgbm_params_opt4 = {'num_leaves': 111,
                    'min_data_in_leaf': 150,
                    'objective':'regression',
                    'max_depth': 9,
                    'learning_rate': 0.001, # 0.002,
                    "boosting": "gbdt",
                    "feature_fraction": 0.7522,
                    "bagging_freq": 1,
                    "bagging_fraction": 0.7083,
                    "bagging_seed": 11,
                    "metric": 'rmse',
                    "lambda_l1": 0.2634,
                    "verbosity": -1,
                    # "nthread": 4,
                    "random_state": 4590} 

# Optimised lgbm params : improvement from fine tuning (best at this moment)
lgbm_params_opt1 = {'num_leaves': 127,
                    'min_data_in_leaf': 148,
                    'objective':'regression',
                    'max_depth': 10,
                    'learning_rate': 0.005, # (best at this moment); 0.001 does not improve
                    "boosting": "gbdt",
                    "feature_fraction": 0.7202,
                    "bagging_freq": 1,
                    "bagging_fraction": 0.8125,
                    "bagging_seed": 11,
                    "metric": 'rmse',
                    "lambda_l1": 0.3468,
                    "random_state": 4590, # 133,
                    "verbosity": -1}

# Optimised lgbm params : check if there is improvement
# Version 41 (177 features) with the following hyperparameter tuning
lgbm_params_opt2 = {'num_leaves':62,
                    'min_data_in_leaf': 20,
                    'objective':'regression',
                    'max_depth': 8,
                    'learning_rate': 0.001,
                    "boosting": "gbdt",
                    "feature_fraction": 0.9,
                    "bagging_freq": 1,
                    "bagging_fraction": 0.9,
                    "bagging_seed": 11,
                    "metric": 'rmse',
                    "lambda_l1": 0.1,
                    "verbosity": -1,
                    # "nthread": 4,
                    "random_state": 4590} 

In [ ]:
# Use by multiple methods and stacking variants with different random_state 
def lgbm_predictions(Xtrain_df, Xtest_df, y, lgbm_params, n_splits):
    categorical_feats = ['feature_1', 'feature_2', 'feature_3']
    num_round = 10000 # 1000
    folds = KFold(n_splits=n_splits, shuffle=True, random_state=4590)

    oof = np.zeros(len(Xtrain_df))
    lgbm_pred = np.zeros(len(Xtest_df))
    start = time.time()
    feature_importance_df = pd.DataFrame()

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(Xtrain_df.values, y.values)):
        print("LGBM fold n°{}".format(fold_))
        print("-" * 80)

        trn_data = lgb.Dataset(Xtrain_df.iloc[trn_idx], # [train_features]
                               label=y.iloc[trn_idx],
                               categorical_feature=categorical_feats)
        val_data = lgb.Dataset(Xtrain_df.iloc[val_idx], # [train_features]
                               label=y.iloc[val_idx],
                               categorical_feature=categorical_feats)

        lgbm_reg = lgb.train(lgbm_params,
                             trn_data,
                             num_round,
                             valid_sets = [trn_data, val_data],
                             verbose_eval=100,
                             early_stopping_rounds = 200)

        oof[val_idx] = lgbm_reg.predict(Xtrain_df.iloc[val_idx], # [train_features]
                                        num_iteration=lgbm_reg.best_iteration)

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = Xtrain_df.columns # train_features
        fold_importance_df["importance"] = lgbm_reg.feature_importance()
        fold_importance_df["fold"] = fold_ + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

        lgbm_pred += lgbm_reg.predict(Xtest_df,
                                      num_iteration=lgbm_reg.best_iteration) / folds.n_splits
    print()
    print("CV RMSE score: {:<8.5f}".format(mean_squared_error(oof, y_nonoutliers)**0.5))
    
    return lgbm_pred

In [ ]:
%%time
import time
# Wall time: 14min 17s (using lgbm_params_opt1)
# Wall time: 1h 4min 11s (using lgbm_params_opt2)
n_splits = 5

# lgbm_params_opt1 gives best public score
lgbm_nonoutliers_pred = lgbm_predictions(X_nonoutliers[train_features], X_test[train_features],
                                         y_nonoutliers, lgbm_params_opt1, n_splits)

lgbm_nonoutliers_pred.shape, X_nonoutliers[train_features].shape

In [ ]:
# Overfits without the outliers data
# Early stopping, best iteration is:
# [1803]	training's rmse: 1.45259	valid_1's rmse: 1.52255
# CV RMSE score: 1.55372 (lgbm opt2) <--
# CV RMSE score: 1.55693 (lgbm opt2)
# CV RMSE score: 1.55512 (lgbm opt1)
# CV RMSE score: 1.55523 (lgbm opt1)
# CV RMSE score: 1.55490 (lgbm opt1, lr=0.001) does not improve

# Early stopping, best iteration is:
# [5203]	training's rmse: 1.4621	valid_1's rmse: 1.52261
# CV RMSE score: 1.55486 (lgbm opt4, lr=0.002)
# Wall time: 37min 2s

# [10000]	training's rmse: 1.46497	valid_1's rmse: 1.52237
# CV RMSE score: 1.55490 (lgbm opt4, lr=0.001)
# Wall time: 1h 5min 16s

# Predicting the outliers correctly should improve on ranking

- ### Predictions made with test set

In [ ]:
# lgbm_predictions_df = lgbm_pred_df['target'].to_frame(), creates dataframe from a pandas.core.series.Series
lgbm_predictions_df = pd.DataFrame(lgbm_pred_df['target'])
lgbm_predictions_df.rename({'target' : 'LGBM_Pred'}, axis=1, inplace=True)

lgbm_nonoutliers_pred_df = pd.DataFrame(lgbm_nonoutliers_pred, columns = ['LGBM_NOutlier_Pred'])
concat_test_pred = pd.concat([lgbm_predictions_df, lgbm_nonoutliers_pred_df], axis=1, sort=False)

concat_test_pred

- ### Perform the average between predictions from LGBM with non-outliers and best single model, LGBM, with complete training data </br>
#### (Average blending 'target' predictions from LGBM trained with non-outliers and complete data set separately)

In [ ]:
# Get the mean value of the 2 ML models
concat_test_pred['Avg_Pred'] = concat_test_pred.mean(axis=1)
concat_test_pred

In [ ]:
X_train[train_features].shape, X_test[train_features].shape

In [ ]:
# Average predicted values from LGBM with non-outliers (>= -33) in target and complete merged dataset
# # Producing worst ranking than single LGBM with Public score : 3.74731
create_file_for_submission("lgbm_opt_blending_nonoutliers_130KSfeat.csv", card_ids, concat_test_pred['Avg_Pred'])

# (B) Method 2 : Build a Classifier to predict outliers
> - #### LGBM Classifier
> - #### LGBM predictions that are > 0.5 will be considered as outliers
> - #### Changing the prediction target values for outliers
> - #### Blend (1) LGBM predictions with (2) LGBM Classifier predicted non-outlier dataset with adjusted 'target

In [ ]:
# Training to predict outliers on test set
# 'outliers' = 1 means the training data instance is an outlier with 'target' < -33
# y is re-assigned 
y = X['outliers']
y.shape

- ### LGBM Classifier

In [ ]:
import lightgbm as lgb

# Basic LGBM classifier
clf = lgb.LGBMClassifier()
clf.fit(X[train_features], y)

y_pred = clf.predict(X_test[train_features])
print("Shape of outlier prediction :", y_pred.shape)

lgbm_outlier_pred_df = pd.DataFrame(y_pred, columns = ['LGBM_Outlier_Pred'])
lgbm_outlier_pred_df.value_counts()

In [ ]:
X[train_features].shape, X_test[train_features].shape, X_train.shape, X_test.shape, X.shape

- ### LGBM predictions that are > 0.5 will be considered as outliers 

In [ ]:
import lightgbm as lgb
# StratifiedKFold for binary or multiclass
from sklearn.model_selection import StratifiedKFold
np.random.seed(7878)

# Best results with params
lgbm_classifier_params = {'num_leaves': 62,
                          'objective':'binary',
                          'metric': 'auc',
                          'max_depth': 8,
                          'learning_rate': 0.01,
                          "boosting_type": "gbdt",
                          "reg_lambda": 0.2634,
                          "random_state": 7878, # 133
                          "verbosity": -1}

In [ ]:
%%time
from sklearn.model_selection import train_test_split

import time
categorical_feats = ['feature_1', 'feature_2', 'feature_3'] # 'feature_1'
num_round = 10000
n_splits= 5

folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=4590)

oof = np.zeros(len(X[train_features]))
lgbm_outlier_pred = np.zeros(len(X_test[train_features]))
start = time.time()
feature_importance_df = pd.DataFrame()

# *** This will replace X_train with only 'train_features'; use X instead ***
X_train, X_valid, y_train, y_valid = train_test_split(X[train_features], y, test_size=0.2, random_state=4590)
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)

params = {'metric' : lgbm_classifier_params['metric'],
          'objective' : lgbm_classifier_params['objective'],
          'verbosity' : lgbm_classifier_params['verbosity'],
          
          # "reg_lambda": lgbm_classifier_params['reg_lambda'],
          # Gives the best AUC (lr=0.01) and ranking with public score : 3.68346
          'learning_rate': lgbm_classifier_params['learning_rate']
          # Gives better AUC 0.90876 but public score : 3.69543
          # 'max_depth' : lgbm_classifier_params['max_depth']
         }  

lgbm_outlier_reg = lgb.train(params, # lgbm_classifier_params,
                             train_data,
                             valid_sets = [valid_data],
                             num_boost_round = num_round,
                             early_stopping_rounds = 100,
                             valid_names=['valid'])

score = lgbm_outlier_reg.best_score['valid']['auc']
print('AUC_score :', score)

In [ ]:
# Early stopping, best iteration is:
# [692]	valid's auc: 0.907677
# AUC_score : 0.9076771007174149
# Wall time: 44.8 s

# With max_depth = 8
# Early stopping, best iteration is:
# [903]	valid's auc: 0.908763
# AUC_score : 0.9087629103180834
# Wall time: 52.4 s

# With 'reg_lambda'
# Early stopping, best iteration is:
# [751]	valid's auc: 0.908012
# AUC_score : 0.9080121760338308
# Wall time: 47.1 s

In [ ]:
# Predict outliers in 'target'
lgbm_outlier_pred = lgbm_outlier_reg.predict(X_test[train_features])
# >= 0.5 will be assign value 1 
lgbm_outlier_pred = lgbm_outlier_pred.round(0)
lgbm_outlier_pred = lgbm_outlier_pred.astype(int)

lgbm_outlier_pred.shape

In [ ]:
lgbm_outlier_pred_df = pd.DataFrame(lgbm_outlier_pred, columns = ['LGBM_Outlier_Pred'])

# Lesser predicted outliers than basic classifier
lgbm_outlier_pred_df.value_counts()

# LGBM_Outlier_Pred without max_depth params
# 0                    123605
# 1                        18

In [ ]:
lgbm_outlier_pred_df["card_id"] = card_ids
lgbm_outlier_pred_df["LGBM_Pred"] = lgbm_pred_df['target'] # pd.Series(lgbm_predictions)
lgbm_outlier_pred_df.head(5)

In [ ]:
# Show only predicted outliers
lgbm_outlier_pred_df.loc[lgbm_outlier_pred_df['LGBM_Outlier_Pred']==1]

In [ ]:
concat_test_pred

In [ ]:
# concat_test_pred['outliers'] = concat_test_pred['LGBM_NOutlier_Pred'] >= concat_test_pred['LGBM_NOutlier_Pred'].quantile(0.9)
# concat_test_pred['outliers'].value_counts()

In [ ]:
# Using the earlier created dataframe for averaging the prediction with non-outlier and complete train data set 
concat_test_pred['LGBM_Outlier_Pred'] = lgbm_outlier_pred
concat_test_pred.loc[concat_test_pred['LGBM_Outlier_Pred']==1]

In [ ]:
concat_test_pred['LGBM_Outlier_Pred'].value_counts()

In [ ]:
# train['final'] = train['bin_predict'](-33.21928)+(1-train['bin_predict'])train['no_outlier']
# Classifier AUC: 0.914; no outlier Regression CV:1.545

# Train with non-outliers with lgbm_params_opt2 -> lgbm_nonoutliers_pred = concat_test_pred['LGBM_NOutlier_Pred']
# Case 1 : Public score : 3.72170 and with LGBM_Pred averaging is 3.69543
# concat_test_pred['Outlier_adj_Pred'] = concat_test_pred['LGBM_Pred'] + \
#                                        concat_test_pred['LGBM_Outlier_Pred']*(-33.219281)

# Case 2 : Public score : 3.68696 and with LGBM_Pred averaging is 3.68706
# concat_test_pred['Outlier_adj_Pred'] = concat_test_pred['LGBM_Pred'] + \
#                                        (concat_test_pred['LGBM_Outlier_Pred'] * concat_test_pred['LGBM_NOutlier_Pred'])

# Case 3 : Public score : 3.70081 and with LGBM_Pred averaging is 3.69074
# concat_test_pred['Outlier_adj_Pred'] = (1 - concat_test_pred['LGBM_Outlier_Pred']) * concat_test_pred['LGBM_Pred'] + \
#                                        (concat_test_pred['LGBM_Outlier_Pred'] * -33.219281)

# Case 4 : Public score : 3.69629 and with LGBM_Pred averaging is 3.68995
# concat_test_pred['Outlier_adj_Pred'] = (1 - concat_test_pred['LGBM_Outlier_Pred']) * concat_test_pred['-33.219281'] + \
#                                        (concat_test_pred['LGBM_Outlier_Pred'] * concat_test_pred['LGBM_NOutlier_Pred'])

# Case 5 : Public score : 3.88467 and with LGBM_Pred averaging is 3.74621
# concat_test_pred['Outlier_adj_Pred'] = (1 - concat_test_pred['LGBM_Outlier_Pred']) * concat_test_pred['LGBM_NOutlier_Pred'] + \
#                                        (concat_test_pred['LGBM_Outlier_Pred'] * -33.219281)

# Train with non-outliers with lgbm_params_opt1 with 'LGBM_NOutlier_Pred' instead of '-33.219281'
# max_depth = 8
# Case 1 : Public score : 3.68690, Private score : 3.61803 and 
# with LGBM_Pred averaging Public score : 3.68703, Private score : 3.61809
# With param 'max_depth' (better)
# ** Case 1 : Public score : 3.68662, Private score : 3.61804 and 
# with LGBM_Pred averaging Public score : 3.68346, Private score : 3.61926 **
concat_test_pred['Outlier_adj_Pred'] = concat_test_pred['LGBM_Pred'] + \
                                       (concat_test_pred['LGBM_Outlier_Pred'] * concat_test_pred['LGBM_NOutlier_Pred'])

concat_test_pred.loc[concat_test_pred['LGBM_Outlier_Pred']==1]

In [ ]:
X.shape

In [ ]:
create_file_for_submission("lgbm_opt2_130KS_feats_KFold5_outliers_opt4_lr2_reg_lambda_adj1.csv",
                           card_ids, concat_test_pred['Outlier_adj_Pred'])

- ### Changing the prediction target values for outliers
> #### final prediction result ＝ (single model LGBM result) + (-33.219281 * binary outlier prediction)

In [ ]:
lgbm_outlier_pred_df

In [ ]:
# Simply assign value to predicted outlier with '-33.219281' and leave predicted values unchanged for non-ouliters
# lgbm_outlier_pred_df.loc[lgbm_outlier_pred_df['LGBM_Outlier_Pred']==1, 'LGBM_Preds'] = -33.219281
# lgbm_outlier_pred_df.loc[lgbm_outlier_pred_df['LGBM_Preds'] == -33.219281]

# The following final result is the different from the above operations by regression result
# final prediction result ＝ (single model LGBM result) + (-33.219281 * binary outlier prediction)
lgbm_outlier_pred_df['Outlier_adj_Pred'] = lgbm_outlier_pred_df['LGBM_Pred'] + \
                                           lgbm_outlier_pred_df['LGBM_Outlier_Pred']*(-33.219281)

print(lgbm_outlier_pred_df.loc[lgbm_outlier_pred_df['LGBM_Outlier_Pred']==1].shape)
lgbm_outlier_pred_df.loc[lgbm_outlier_pred_df['LGBM_Outlier_Pred']==1]

In [ ]:
concat_test_pred

In [ ]:
concat_test_pred['Outlier_adj_Pred'] = lgbm_outlier_pred_df['Outlier_adj_Pred']
concat_test_pred.loc[concat_test_pred['LGBM_Outlier_Pred']==1]

- #### Blend (1) LGBM predictions with (2) LGBM Classifier predicted non-outlier dataset with adjusted 'target'

In [ ]:
concat_test_pred['Avg_Pred'] = concat_test_pred[['LGBM_Pred', 'Outlier_adj_Pred']].mean(axis=1)

print(concat_test_pred.loc[concat_test_pred['LGBM_Outlier_Pred']==1].shape)
concat_test_pred.loc[concat_test_pred['LGBM_Outlier_Pred']==1]

In [ ]:
# Producing better ranking than single LGBM with
# Private score : 3.61926 (1637, 39.83%), Public score : 3.68346 (329, 8%)
create_file_for_submission("lgbm_opt2_130KS_feats_KFold5_outliers_opt1_lr0_01_lgbmc_lr0_02_adj2_avg.csv",
                           card_ids, concat_test_pred['Avg_Pred'])

# (C) Method 3 : Blend predictions
> #### 1. LGBM trained all data instances from train dataset
> #### 2. LGBM trained data instances predicted as non-outliers from train dataset (Outlier Classifier)
> #### 3. Blending with mean of 2 above models

In [ ]:
# X_train replaced with only relevant 'train_features', use 'X' instead
# X_train['outliers'].value_counts()
X.loc[X['target'] < -33, 'target'].value_counts()

In [ ]:
X.shape

In [ ]:
# Preparation for Layer 3 : Blending model (Mean)
# Predict outliers in 'target' for train dataset; change values in X_train['outliers']
X['outliers'] = lgbm_outlier_reg.predict(X[train_features])
# >= 0.5 will be assign value 1 
X['outliers'] = X['outliers'].round(0)
X['outliers'] = X['outliers'].astype(int)

# Predicted 'outliers' are less than 2,207 ('target' < -33)
X['outliers'].value_counts()

In [ ]:
# Show only predicted outliers
X.loc[X['outliers']==1]

In [ ]:
# 201554 + 363
# 201824 + 93 (LGBM classifier with better AUC Score)
X.shape

In [ ]:
# Getting only the predicted non-outliers (non-assigned outlier with 'outliers' column == 0)
X_nonoutliers = X.loc[X['outliers'] == 0]
y_nonoutliers = X.loc[X['outliers'] == 0, 'target']
X_nonoutliers.shape, X_nonoutliers[train_features].shape, y_nonoutliers.shape

In [ ]:
X_nonoutliers[train_features].shape, X_test[train_features].shape, y_nonoutliers.shape

In [ ]:
%%time
import time

n_splits = 5
lgbm_nonoutliers_pred = lgbm_predictions(X_nonoutliers[train_features], X_test[train_features],
                                         y_nonoutliers, lgbm_params_opt1, n_splits) # lgbm_params_opt2

lgbm_nonoutliers_pred.shape
# CV RMSE score: 3.54294 (lgbm_params_opt1 with 10K rounds is better than lgbm_params_opt2 with 10K rounds)
# CV RMSE score: 3.54258 (lgbm_params_opt1 with 10K rounds is best)
# CV RMSE score: 3.54606 (lgbm_params_opt2 with 10K rounds is better than lgbm_params_opt2 with 1K rounds)

# Early stopping, best iteration is:
# [1197]	training's rmse: 3.33551	valid_1's rmse: 3.67731
# CV RMSE score: 3.61183 
# Wall time: 8min 15s

- #### Blending with mean of 2 above models

In [ ]:
# Load best ranked predictions from output directory in current session
# lgbm_pred_df = pd.read_csv('lgbm_opt2_130KS_merid_puramt_date_histmer_rdiff_w2v_installments_K5.csv')
lgbm_pred_df = pd.read_csv('../input/test-predictions/lgbm_opt2_130KSfeats_K5.csv')

lgbm_nonoutliers_pred_df = pd.DataFrame()
lgbm_nonoutliers_pred_df["card_id"] = card_ids
lgbm_nonoutliers_pred_df["LGBM_Pred"] = lgbm_pred_df['target'] # pd.Series(lgbm_predictions)

lgbm_nonoutliers_pred_df['LGBM_NonOutliers_Pred'] = lgbm_nonoutliers_pred

# Lesser predicted outliers than basic classifier
lgbm_nonoutliers_pred_df

In [ ]:
lgbm_nonoutliers_pred_df['Avg_Pred'] = lgbm_nonoutliers_pred_df[['LGBM_Pred', 'LGBM_NonOutliers_Pred']].mean(axis=1)
lgbm_nonoutliers_pred_df

In [ ]:
# train['final'] = train['bin_predict'](-33.21928)+(1-train['bin_predict'])train['no_outlier']

In [ ]:
# Using best 130KS features with opt2, KFold5
# Producing worst ranking than single LGBM with Public score : 3.69549 but better than above outlier methods 1 and 2
create_file_for_submission("lgbm_opt2r10K_130KS_feats_KFold5_lgbmc_nonoutliers_avg.csv",
                           card_ids, lgbm_nonoutliers_pred_df['Avg_Pred'])

In [ ]:
# Training based on 'target' predictions from layer 1 models with ground truth on train dataset

# (D) Method 4 : Blending with Bayesian Ridge
> #### 1. LGBM trained all data instances from train dataset
> #### 2. LGBM trained data instances predicted as non-outliers from train dataset (Outlier Classifier)
> #### 3. Blending with mean of 2 above models (Method 3)

In [ ]:
%%time
from sklearn.linear_model import BayesianRidge

bayesridge_clf = BayesianRidge(compute_score=True)
bayesridge_clf.fit(concat_train_pred, y)

In [ ]:
# BayesianRidge prediction based on blending
bayesridge_pred = bayesridge_clf.predict(lgbm_nonoutliers_pred_df[['LGBM_Pred', 'LGBM_NonOutliers_Pred', 'Avg_Pred']])
bayesridge_pred.shape

In [ ]:
# for f in ['feature_1','feature_2','feature_3']:
#     # Setting mean value of the 'outliers' for the input features 
#     order_label = X_train.groupby([f])['outliers'].mean()
#     X_train[f] = X_train[f].map(order_label)
#     X_test[f] = X_test[f].map(order_label)

# X_train.loc[0:5, ['feature_1','feature_2','feature_3']]

# (E) Method 5 : Stacking with Blending
> #### 1. Load trained single machine learning models of LGBM, XGBR, RF and DL in first layer
> #### 2. Using Decision Tree Regressor and LGBM in second layer
> #### 3. Final layer with average blending

In [ ]:
X[train_features].shape, y.shape

In [ ]:
!ls -tl ../input/test-predictions

- #### Load from saved predictions from different models

In [ ]:
# Load model predictions trained with 130 KS features
lgbm_pred = pd.read_csv('../input/test-predictions/lgbm_opt2_130KSfeats_K5.csv')
xgbr_pred = pd.read_csv('../input/test-predictions/xgbr_130KSfeats_K5.csv')
rf_pred = pd.read_csv('../input/test-predictions/rf_130KS_feats.csv')
dl_pred = pd.read_csv('../input/test-predictions/DL3_v1_130KSfeats_adam.csv')

test_pred = pd.concat([lgbm_pred['target'], xgbr_pred['target'],
                       rf_pred['target'], dl_pred['target']],
                      axis=1, keys=['LGBM_Pred', 'XGBR_Pred', 'RF_Pred', 'DL_Pred'])

# dl_preds = pd.read_csv('../input/testpredictions/xgbr_lgbm_dl_79feat.csv')
# concat_pred_df = pd.concat([xgbr_preds['target'], lgbm_preds['target'], dl_preds['target']],
#                             axis=1, keys=['XGBR_Pred', 'LGBM_Pred', 'DL_Pred'])

# concat_pred_df['Avg_Pred'] = concat_pred_df.mean(axis=1)

# xgbr_preds.loc[:5, 'target'], lgbm_preds.loc[:5, 'target'], dl_preds.loc[:5, 'target']

# concat_test_pred.drop('Avg_Pred', axis=1)
# test_pred = concat_test_pred[['LGBM_1_Pred', 'RF_Pred', 'DL_Pred']]
# test_pred = concat_test_pred
test_pred

In [ ]:
# Load the saved Light Gradient Boosting model
lgbm_reg = lgb.Booster(model_file='../input/test-predictions/lgbm_optmodel2_130KSfeats_K5_3.68718.txt')  # init model
lgbm_reg.params

- #### Load trained single machine learning models of LGBM, XGBR, RF and DL in first layer
> #### (from shared Kaggle dataset, preloaded in Kaggle Notebook, please contact Jit Seah to have access)

In [ ]:
# For loading XGBR model
import os 
import xgboost as xgb

#### If XGBRegressor model exists, load it
xgbr_model_file = '../input/test-predictions/xgbr_130KSfeats_K5.json'

if os.path.exists(xgbr_model_file):
    # xgb_Classifier.save_model and load_model give an "le" error when trying to obtain score
    # Retrieve model file if exist so that training do not need to done
    xgbr_model = xgb.XGBRegressor()
    print("Unpickling existing XGBRegressor model...")
    xgbr_model.load_model(xgbr_model_file)
    
    # xgb_model.get_xgb_params()
    print("Loaded model :\n", xgbr_model)
    print("with type\n", type(xgbr_model))
    
# Check if there is any difference in the model saved and loaded predictions
# There are differences in predictions from loaded model
xgbr_predictions_new = xgbr_model.predict(X_test[train_features])
pred_diff = np.setdiff1d(xgbr_pred['target'], xgbr_predictions_new)
len(pred_diff), xgbr_pred['target'].shape

In [ ]:
# For loading DL model in h5 format
from keras.models import load_model

# Load the saved Deep Learning model
dl_model = load_model('../input/test-predictions/DL3_v1_130KSfeats_adam.h5')
dl_model.summary()

# Check if there is any difference in the model saved and loaded predictions
# There are NO differences in predictions from loaded model
dl_predictions_new = dl_model.predict(X_test[train_features])
pred_diff = np.setdiff1d(dl_pred['target'], dl_predictions_new)
len(pred_diff), dl_pred['target'].shape

In [ ]:
# For loading Random Forest Regressor model
# forest_reg = load_model_from_picklefile('rf_1KEst_130KS_feats_pickle')
forest_reg = joblib.load("../input/test-predictions/rf_1KEst_130KSfeats.joblib")
print("RF params :", forest_reg.get_params)

# Check if there is any difference in the model saved and loaded predictions
# There are differences in predictions from loaded model
rf_predictions_new = forest_reg.predict(X_test[train_features])
pred_diff = np.setdiff1d(rf_pred['target'], rf_predictions_new)
len(pred_diff), rf_pred['target'].shape

In [ ]:
y.shape, X_train[train_features].shape, X_test[train_features].shape

#### 1. Load trained models of LGBM, XGBR, RF and DL in first layer

In [ ]:
%%time
lgbm_train_pred = lgbm_reg.predict(X[train_features])
xgbr_train_pred = xgbr_model.predict(X[train_features])
rf_train_pred = forest_reg.predict(X[train_features])
dl_train_pred = dl_model.predict(X[train_features])
# dtreg_train_pred = tree_reg.predict(X[train_features])

lgbm_train_pred_df = pd.DataFrame(lgbm_train_pred, columns = ['LGBM_Tr_Pred'])
xgbr_train_pred_df = pd.DataFrame(xgbr_train_pred, columns = ['XGBR_Tr_Pred'])
rf_train_pred_df = pd.DataFrame(rf_train_pred, columns = ['RF_Tr_Pred'])
dl_train_pred_df = pd.DataFrame(dl_train_pred, columns = ['DL_Tr_Pred'])
# dtreg_train_pred_df = pd.DataFrame(dtreg_train_pred, columns = ['DTR_Tr_Pred'])

# xgbr_train_pred_df, lgbm_train_pred_df, rfb_train_pred_df and dl_train_pred_df
train_pred = pd.concat([lgbm_train_pred_df, xgbr_train_pred_df, rf_train_pred_df, dl_train_pred_df],
                       axis=1, sort=False)

train_pred

In [ ]:
train_pred.shape, test_pred.shape

#### 2. Using Random Forest Regressor and LGBM in second layer

In [ ]:
# %%time
# Linear Regression model to have ensemble regression
# from sklearn.tree import DecisionTreeRegressor
# Cost : Root Mean Square Error, RMSE
# from sklearn.metrics import mean_squared_error

# tree_reg = DecisionTreeRegressor()
# tree_reg.fit(train_pred, y)

# train_pred = tree_reg.predict(train_pred)
# tree_mse = mean_squared_error(y, train_pred)
# tree_rmse = np.sqrt(tree_mse)
# print("DecisionTreeRegressor RMSE :", tree_rmse)

# scores = cross_val_score(tree_reg, concat_train_pred, y,
#                          scoring="neg_mean_squared_error", cv=10)

# tree_rmse_scores = np.sqrt(-scores) # opposite of MSE, need to have negative sign

# # Different results when it is executed
# display_scores(tree_rmse_scores)

# SL2_tree_reg_pred = tree_reg.predict(test_pred)
# SL2_tree_reg_pred.shape

# Using a trained linear regressor with predicted values from XGBR, LGBM, RF and DL
# Overfits and giving poor tree reg stacking results, Public score : 4.91414
# create_file_for_submission("lgbm_xgbr_rf_dl_130KSfeat_SL2_tree_reg.csv", card_ids, SL2_tree_reg_pred)

In [ ]:
%%time
# Random forest model as part of ensemble regressor
from sklearn.ensemble import RandomForestRegressor

# The default value of n_estimators will change from 10 in version 0.20 to 100 in version 0.22
SL2_forest_reg = RandomForestRegressor(n_estimators = 1000,
                                       max_depth = 9, # 15
                                       min_samples_leaf = 20, # 149
                                       min_weight_fraction_leaf = 0.2,
                                       bootstrap = True,
                                       oob_score = True,
                                       n_jobs = -1,
                                       random_state = 100,
                                       verbose=1)

SL2_forest_reg.fit(train_pred, y)

data_predictions = SL2_forest_reg.predict(train_pred)
SL2_forest_reg_mse = mean_squared_error(y, data_predictions)
SL2_forest_reg_rmse = np.sqrt(SL2_forest_reg_mse)

print("RandomForestRegressor RMSE :", forest_rmse)

In [ ]:
SL2_rf_reg_pred = SL2_forest_reg.predict(test_pred)
SL2_rf_reg_pred.shape

In [ ]:
# Using a RF regressor with predicted values from XGBR, LGBM, RF and DL
# Overfits and doing better than tree reg, Public score : 3.93717
create_file_for_submission("lgbm_xgbr_rf_dl_130KSfeat_SL2_rf_reg.csv", card_ids, SL2_rf_reg_pred)

In [ ]:
%%time
import time

num_round = 10000
n_splits=5

folds = KFold(n_splits=n_splits, shuffle=True, random_state=4590) # 27
oof = np.zeros(len(train_pred))
SL2_lgbm_pred = np.zeros(len(test_pred))
start = time.time()
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_pred.values, y.values)):
    print("LGBM fold n°{}".format(fold_))
    print("-" * 80)
    
    trn_data = lgb.Dataset(train_pred.iloc[trn_idx], # [train_features]
                           label=y.iloc[trn_idx])
    val_data = lgb.Dataset(train_pred.iloc[val_idx], # [train_features]
                           label=y.iloc[val_idx])

    # lgbm_params_opt2 performs better than lgbm_params_opt1 andlgbm_params 
    SL2_lgbm_reg = lgb.train(lgbm_params_opt2,
                             trn_data,
                             num_round,
                             valid_sets = [trn_data, val_data],
                             verbose_eval=100,
                             early_stopping_rounds = 200)
    
    oof[val_idx] = SL2_lgbm_reg.predict(train_pred.iloc[val_idx], # [train_features]
                                        num_iteration=SL2_lgbm_reg.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = train_pred.columns
    fold_importance_df["importance"] = SL2_lgbm_reg.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    SL2_lgbm_pred += SL2_lgbm_reg.predict(test_pred,
                                          num_iteration=SL2_lgbm_reg.best_iteration) / folds.n_splits
    print()
    
    # Save model
    # lgbm_pred_reg.save_model('lgbm_pred_reg_model.txt', num_iteration=bst.best_iteration)

print("CV RMSE score: {:<8.5f}".format(mean_squared_error(oof, y)**0.5))

In [ ]:
# Wall time: 28.1 s (using lgbm_params_opt1)
# CV RMSE score: 3.84539
# Wall time: 1min 40s (using lgbm_params_opt2)
# CV RMSE score: 3.84296
# Wall time: 27.7 s (using lgbm_params)
# CV RMSE score: 3.84527
# lgbm_params_opt2 gives better public score

In [ ]:
# Early stopping, best iteration is:
# [3656]	training's rmse: 2.27884	valid_1's rmse: 2.28246
# CV RMSE score: 2.33106
# Wall time: 5min 6s

In [ ]:
train_pred.shape, SL2_lgbm_pred.shape

In [ ]:
# Using a LGBM with predicted values from XGBR, LGBM, RF and DL
# Overfits and giving poor LGBM stacking results with opt2, Public score : 3.92293 (3.87041)
# Overfits and giving poor LGBM stacking results with opt1, Public score : 3.93402
# Overfits and giving poor LGBM stacking results with lgbm_params, Public score : 3.93414
create_file_for_submission("lgbm_xgbr_rf_dl_130KSfeat_SL2_lgbm.csv", card_ids, SL2_lgbm_pred)

In [ ]:
%%time
from sklearn.linear_model import BayesianRidge

bayesridge_clf = BayesianRidge(compute_score=True)
bayesridge_clf.fit(train_pred, y)

SL2_bayesridge_pred = bayesridge_clf.predict(test_pred)
SL2_bayesridge_pred.shape

In [ ]:
# Using a trained linear regressor with predicted values from XGBR, LGBM, RF and DL
# Overfits but giving better stacking results than stack 2 LGBM and Tree Reg, Public score : 3.72625
create_file_for_submission("lgbm_xgbr_rf_dl_130KSfeat_SL2_bayes_ridge.csv", card_ids, SL2_bayesridge_pred)

#### 3. Final layer with average blending

In [ ]:
# Stacking layer No. 3
SL3_rf_reg_test_pred = pd.DataFrame(SL2_rf_reg_pred, columns = ['ESM_RF_Pred'])
SL3_lgbm_test_pred = pd.DataFrame(SL2_lgbm_pred, columns = ['ESM_LGBM_Pred'])
SL3_bayesr_test_pred = pd.DataFrame(SL2_bayesridge_pred, columns = ['ESM_BAYESR_Pred'])

ensemble_test_pred = pd.concat([SL3_rf_reg_test_pred, SL3_lgbm_test_pred, SL3_bayesr_test_pred],
                               axis=1, sort=False)
ensemble_test_pred['Avg_Blend_Pred'] = ensemble_test_pred.mean(axis=1)

ensemble_test_pred

In [ ]:
ensemble_test_pred['Avg_Blend_Pred'].shape

In [ ]:
# Using a trained linear regressor with predicted values from XGBR, LGBM, RF and DL
# Overfits but giving better stacking results than stack 2 Tree Reg, Public score : 3.92238
create_file_for_submission("lgbm_xgbr_rf_dl_130KSfeat_SL3_avg_blend.csv", card_ids,
                           ensemble_test_pred['Avg_Blend_Pred'])

# (F) Method 6 : Stacking with BayesianRidge in final Blending
> #### 1. Load trained single machine learning models of LGBM, XGBR, RF and DL in first layer
> #### 2. Using Random Forest Regressor and LGBM in second layer
> #### 3. Final layer with BayesianRidge

In [ ]:
train_pred.shape, test_pred.shape

In [ ]:
SL2_rf_train_pred = SL2_forest_reg.predict(train_pred)
SL2_lgbm_train_pred = SL2_lgbm_reg.predict(train_pred)

SL2_rf_train_pred_df = pd.DataFrame(SL2_rf_train_pred, columns = ['SL2_RF_Tr_Pred'])
SL2_lgbm_train_pred_df = pd.DataFrame(SL2_lgbm_train_pred, columns = ['SL2_LGBM_Tr_Pred'])

SL2_train_pred = pd.concat([SL2_lgbm_train_pred_df, SL2_rf_train_pred_df],
                           axis=1, sort=False)

SL2_train_pred

In [ ]:
SL2_rf_test_pred = pd.DataFrame(SL2_rf_reg_pred, columns = ['SL2_RF_Pred'])
SL2_lgbm_test_pred = pd.DataFrame(SL2_lgbm_pred, columns = ['SL2_LGBM_Pred'])

SL2_test_pred = pd.concat([SL2_rf_test_pred, SL2_lgbm_test_pred], axis=1, sort=False)
SL2_test_pred

In [ ]:
SL2_train_pred.shape, SL2_test_pred.shape

In [ ]:
# Stacking layer No. 3
SL3_test_pred = SL2_test_pred
SL3_test_pred['Avg_Blend_Pred'] = SL2_test_pred.mean(axis=1)
SL3_test_pred

In [ ]:
# Using a trained linear regressor with predicted values from XGBR, LGBM, RF and DL
# Overfits but giving better stacking results than stack 2 Tree Reg, Public score : 3.91828
create_file_for_submission("lgbm_xgbr_rf_dl_130KSfeat_SL2_rf_lgbm_SL3_avg_blend.csv", card_ids,
                           SL3_test_pred['Avg_Blend_Pred'])

In [ ]:
%%time
from sklearn.linear_model import BayesianRidge

# Stacking layer No.3 : BayesianRidge on layer 2 models (RF, LGBM)
bayesridge_clf = BayesianRidge(compute_score=True)
bayesridge_clf.fit(SL2_train_pred, y)

SL3_bayesridge_pred = bayesridge_clf.predict(SL2_test_pred)
SL3_bayesridge_pred.shape

In [ ]:
# Using a trained linear regressor with predicted values from XGBR, LGBM, RF and DL
# Overfits but giving better stacking results than stack 2 Tree Reg, Public score : 3.91297
create_file_for_submission("lgbm_xgbr_rf_dl_130KSfeat_SL3_bayesridge.csv", card_ids, SL3_bayesridge_pred)

# Part 3 : Single Baseline Models

In [ ]:
list_discarded

- #### Exclude unwanted features for training without recreating the merged data

In [ ]:
X_train[train_features].shape, X_test[train_features].shape

In [ ]:
# Remove features in attempt to get better CV RMSE scores
print("No. of train_features :", len(train_features))

old_list_discarded = ['hist_purchase_amount_max', 'hist_category_3_sum_purchase_amount_mean', 'hist_installments_mean']
features_to_remove = list_discarded
# features_to_remove = ['new_merchant_purchase_amount_sum', 'new_merchant_merchants_most_recent_purchases_range_min_max',
#                       'new_merchant_merchants_avg_purchases_lag3_sum_min', 'new_merchant_merchants_avg_sales_lag6_mean_max',
#                       'new_merchant_merchants_most_recent_sales_range_max_min', 'new_merchant_w2v_new_state_id_0_std',
#                       'new_merchant_card_id_size', 'new_merchant_w2v_new_state_id_1_std',
#                       'new_merchant_merchants_avg_sales_lag3_mean_max', 'new_merchant_merchants_avg_purchases_lag6_sum_max']

train_features = list(set(train_features) - set(features_to_remove))

print("No. of train_features after removing some low important features :", len(train_features))

len(set(features_to_remove)), len(set(train_features))

In [ ]:
# Cost : Root Mean Square Error, RMSE
from sklearn.metrics import mean_squared_error
# Better Evaluation using cross-validation
from sklearn.model_selection import cross_val_score

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard Deviation:", scores.std())

# (A) Random Forest Regressor

In [ ]:
X_train[train_features].shape

In [ ]:
%%time
# Ensemble Learning : RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor

# The default value of n_estimators will change from 10 in version 0.20 to 100 in version 0.22
forest_reg = RandomForestRegressor(n_estimators = 1000, # 1000
                                   max_depth = 9, # 15
                                   min_samples_leaf = 20, # 149
                                   min_weight_fraction_leaf = 0.2,
                                   bootstrap = True,
                                   oob_score = True, n_jobs = -1,
                                   random_state = 100,
                                   verbose=1)

forest_reg.fit(X_train[train_features], X_train['target'])

data_predictions = forest_reg.predict(X_train[train_features])
forest_mse = mean_squared_error(X_train['target'], data_predictions)
forest_rmse = np.sqrt(forest_mse)

print("RandomForestRegressor RMSE :", forest_rmse)

# forest_scores = cross_val_score(forest_reg, X_train[train_features], X_train['target'],
#                                 scoring="neg_mean_squared_error", cv=10)

# forest_rmse_scores = np.sqrt(-forest_scores) # opposite of MSE, need to have negative sign

# display_scores(forest_rmse_scores)

# Scores: [3.90089189 4.04644846 3.99770616 3.98337336 4.00308421 3.89694652
#  3.9277469  4.12725579 4.00782351 4.04242215]
# Mean: 3.993369895502748
# Standard Deviation: 0.06762168624862287
# Wall time: 14min 53s

# RandomForestRegressor RMSE : 3.7898758159311647 (194 features with PriSc: 3.75530, PubSc: 3.86015)
# RandomForestRegressor RMSE : 3.7979328113935664 (130KS features with PriSc: 3.76409, PubSc: 3.86907)

In [ ]:
# forest_reg.oob_score_
forest_reg.score(X_train[train_features], X_train['target'])
# max_depth = 9 or 15 RandomForestRegressor gives same RMSE : 3.7979328113935664
# forest_reg.score is 0.02711288566923864

In [ ]:
rf_predictions = forest_reg.predict(X_test[train_features])
rf_predictions.shape

In [ ]:
X_train[train_features].shape

In [ ]:
# Save the model
# save_model_to_picklefile('rf_1KEst_130KS_feats_pickle', forest_reg)
joblib.dump(forest_reg, "rf_1KEst_130KSfeats.joblib")
del forest_reg

In [ ]:
# Load the saved model
# forest_reg = load_model_from_picklefile('rf_1KEst_130KS_feats_pickle')
forest_reg = joblib.load("rf_1KEst_130KS_feats.joblib")
print("RF params :", forest_reg.get_params)

# Check if there is any difference in the model saved and loaded predictions
# There are differences in predictions from loaded model
rf_predictions_new = forest_reg.predict(X_test[train_features])
pred_diff = np.setdiff1d(rf_predictions, rf_predictions_new)

len(pred_diff)

In [ ]:
create_file_for_submission("rf_130KS_feats.csv", X_test["card_id"], rf_predictions)

In [ ]:
%%time
# Search the best combination of hyperparameter values
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [100, 200], 'max_features': [80, 120, X_train[train_features].shape[1]]}, # 'max_features': [2, 4, 6, 8]
    {'bootstrap': [False], 'n_estimators': [100, 200], 'max_features': [100, X_train[train_features].shape[1]]}, # 'max_features': [2, 3, 4]
]

# The default value of n_estimators will change from 10 in version 0.20 to 100 in version 0.22
forest_reg = RandomForestRegressor(n_estimators=200) # no more warning on default 'n_estimators' value

grid_search = GridSearchCV(forest_reg, param_grid, cv=7, scoring="neg_mean_squared_error")

grid_search.fit(X_train[train_features], y)

# CPU times: user 46min 45s, sys: 4.79 s, total: 46min 50s
# Wall time: 46min 50s

# GridSearchCV(cv=5, estimator=RandomForestRegressor(n_estimators=10),
#              param_grid=[{'max_features': [10, 15, 20],
#                           'n_estimators': [3, 10, 30]},
#                          {'bootstrap': [False], 'max_features': [9, 11, 17],
#                           'n_estimators': [3, 10]}],
#              scoring='neg_mean_squared_error')

# grid_search.best_params_
# {'max_features': 15, 'n_estimators': 30}

# GridSearchCV(cv=5, estimator=RandomForestRegressor(n_estimators=10),
#              param_grid=[{'max_features': [10, 15, 20, 30, 37],
#                           'n_estimators': [3, 10, 30]},
#                          {'bootstrap': [False],
#                           'max_features': [9, 11, 17, 27, 31],
#                           'n_estimators': [3, 10]}],
#              scoring='neg_mean_squared_error')

# grid_search.best_params_
# {'max_features': 10, 'n_estimators': 30}

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

In [ ]:
cv_result = grid_search.cv_results_
for mean_score, params in zip(cv_result["mean_test_score"], cv_result["params"]):
    print(np.sqrt(-mean_score), params) # negative mean_score
    
# RMSE : 3.745822393200406 {'max_features': 10, 'n_estimators': 30} with 85 features

In [ ]:
# Analyze the Best Models and their Errors
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

In [ ]:
len(feature_importances)

In [ ]:
# Using GridSearchCV to obtain final model with best estimator
# final_model = grid_search.best_estimator_
rf_best = grid_search.best_estimator_

In [ ]:
brf_predictions = rf_best.predict(X_test[train_features])
brf_predictions.shape

In [ ]:
X[train_features].shape

In [ ]:
create_file_for_submission("brf_138_feats.csv", card_ids, brf_predictions)

In [ ]:
# from sklearn.linear_model import LinearRegression

# # Linear Regression model
# linear_reg = LinearRegression()
# linear_reg.fit(X_train, data_labels)

# # first_batchdata = X_train.iloc[:5]
# # first_batchlabels = data_labels.iloc[:5]
# # print("Predictions: ", linear_reg.predict(first_batchdata))
# # print("Labels: ", list(first_batchlabels))

# data_predictions = linear_reg.predict(X_train)
# linear_mse = mean_squared_error(data_labels, data_predictions)
# linear_rmse = np.sqrt(linear_mse)

# linear_rmse # underfitting data; high bias

# %%time
# # More Powerful Algorithm : DecisionTreeRegressor
# from sklearn.tree import DecisionTreeRegressor

# tree_reg = DecisionTreeRegressor()
# tree_reg.fit(X_train[train_features], X_train['target'])

# tree_reg_predictions = tree_reg.predict(X_train[train_features])
# tree_mse = mean_squared_error(X_train['target'], tree_reg_predictions)
# tree_rmse = np.sqrt(tree_mse)

# print("DecisionTreeRegressor RMSE :", tree_rmse)

# scores = cross_val_score(tree_reg, X_train, data_labels,
#                          scoring="neg_mean_squared_error", cv=10)

# tree_rmse_scores = np.sqrt(-scores) # opposite of MSE, need to have negative sign

# # Different results when it is executed
# display_scores(tree_rmse_scores)

# card_ids = X_test["card_id"].copy()
# tree_reg_predictions = tree_reg.predict(X_test[train_features])

# create_file_for_submission("dtreereg_85_feats.csv", card_ids, tree_reg_predictions)

# (B) eXtreme Gradient Boosting Regressor (XGBR)
> https://xgboost.readthedocs.io/en/latest/parameter.html#

In [ ]:
X_train.shape, X_test.shape

In [ ]:
# Check for missing values : X_train.isna().any()
check_missing_values(X_train), check_missing_values(X_test)

In [ ]:
# Prepare data for training and prediction
X = X_train.copy()
y = X['target']
card_ids = X_test["card_id"].copy()

X.shape, y.shape, X_train.shape, X_test.shape

In [ ]:
# Check for intersection between train features and missing values
intersect_features = set(train_features).intersection(check_missing_values(X_test)) # df_train
print("Common features in train and missing value test features (should be 0): {}".format(len(intersect_features)))
intersect_features

In [ ]:
# Difference in card_ids
col_difference = set(X.columns).symmetric_difference(X_test.columns)
print("Difference in train and test features: {}".format(len(col_difference)))
col_difference

In [ ]:
np.isinf(X[train_features]).any().value_counts()

# print("printing column name where infinity is present")
# col_name = X[train_features].columns.to_series()[np.isinf(X[train_features]).any()]
# print(col_name)

In [ ]:
np.isinf(X_test[train_features]).any().value_counts()

In [ ]:
X.shape, y.shape, X_train.shape, X_test.shape

In [ ]:
%%time
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

import xgboost as xgb
np.random.seed(4590) # 2728
n_splits = 5

kfolds = KFold(n_splits=n_splits, shuffle=True, random_state=4590) # 2018

print("kfolds :", kfolds)
# Make importance dataframe
importances = pd.DataFrame()

oof_preds = np.zeros(X.shape[0])
sub_preds = np.zeros(X_test.shape[0])

## timeit will cause problem with importance
for n_fold, (trn_idx, val_idx) in enumerate(kfolds.split(X[train_features], y)): # X
    X_train_kf, y_train_kf = X[train_features].iloc[trn_idx], y.iloc[trn_idx]
    X_valid_kf, y_valid_kf = X[train_features].iloc[val_idx], y.iloc[val_idx]
    
    print("XGBR fold :", n_fold)
    print("=" * 80)
    
    # XGBoost Regressor estimator
    xgb_model = xgb.XGBRegressor(
        max_depth = 10, # 15
        learning_rate = 0.01,
        n_estimators = 1000,
        subsample = .9,
        colsample_bylevel = .9,
        colsample_bytree = .9,
        min_child_weight= .9,
        gamma = 0,
        random_state = 4590, # 100
        booster = 'gbtree',
        objective = 'reg:squarederror' # 'reg:linear' deprecated
    )
    
    # Training
    xgb_model.fit(
        X_train_kf, y_train_kf,
        eval_set=[(X_train_kf, y_train_kf), (X_valid_kf, y_valid_kf)],
        verbose=True, eval_metric='rmse',
        early_stopping_rounds=100
    )
    
    # Feature importance
    imp_df = pd.DataFrame()
    imp_df['feature'] = train_features
    imp_df['gain'] = xgb_model.feature_importances_
    imp_df['fold'] = n_fold + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)
    
    oof_preds[val_idx] = xgb_model.predict(X_valid_kf)
    test_preds = xgb_model.predict(X_test[train_features])
    sub_preds += test_preds / kfolds.n_splits
    
    print("Next fold :", n_fold+1)
    print()
    
print("Final RMSE : ", np.sqrt(mean_squared_error(y, oof_preds)))

#### Validations with RMSE
> [391]	validation_0-rmse:1.51994	validation_1-rmse:3.73630</br>
> Final RMSE :  3.6854681990466274</br>

> [385]	validation_0-rmse:1.61745	validation_1-rmse:3.73726</br>
> Final RMSE :  3.687870975295424</br>
> Wall time: 1h 19min 11s</br>

> [445]	validation_0-rmse:1.52606	validation_1-rmse:3.72377</br>
> KFold : 5</br>
> Final RMSE :  3.6762817708713555</br>
> Wall time: 2h 30min 50s</br>

> 130KS features
> [709]	validation_0-rmse:2.45425	validation_1-rmse:3.64890</br>
> Next fold : 5</br>
> Final RMSE :  3.662147988178499</br>
> Wall time: 1h 14min 59s</br>
> Private score 3.62528 ; Public score : 3.69528</br>
--</br>
**Reference timing:**</br>
[721]	validation_0-rmse:1.21498	validation_1-rmse:3.88689</br>
Next fold : 17</br>
Final RMSE :  3.6687819164983986</br>
**Wall time: 6h 23min 7s**</br>

In [ ]:
X_train[train_features].shape

#### Save trained ML models

In [ ]:
# https://xgboost.readthedocs.io/en/latest/python/python_api.html?highlight=save_model#xgboost.XGBRegressor.save_model
# enable JSON format support for model IO (saving only the trees and objective)
xgb_model.save_model('xgbr_130KSfeats_K5.json')

In [ ]:
# Pickling files with pickle.dump, then the model may not be accessible in later versions of XGBoost
print("Pickling one of the XGBRegressor model...")
filename = './XGBRegressor_model_pickle'
save_model_to_picklefile(filename, xgb_model)

#### Feature Importances

In [ ]:
importances['gain_log'] = importances['gain']
mean_gain = importances[['gain', 'feature']].groupby('feature').mean()
importances['mean_gain'] = importances['feature'].map(mean_gain['gain'])

plt.figure(figsize=(16, 24))
sns.barplot(x='gain_log', y='feature', data=importances.sort_values('mean_gain', ascending=False))

plt.title('XGBRegressor Features (avg over folds)')
plt.tight_layout()
plt.savefig('xgbr_importances_126feats_K17.png')

In [ ]:
# Length of submission
len(sub_preds), n_fold

In [ ]:
xgbr_predictions = sub_preds
xgbr_predictions.shape

In [ ]:
X[train_features].shape

In [ ]:
create_file_for_submission("xgbr_130KSfeats_K5.csv", card_ids, xgbr_predictions)

In [ ]:
import os 

#### If XGBRegressor model exists, load it
xgbr_model_file = '../input/testpredictions/xgbr_130KSfeats_K5.json'

if os.path.exists(xgbr_model_file):
    # xgb_Classifier.save_model and load_model give an "le" error when trying to obtain score
    # Retrieve model file if exist so that training do not need to done
    xgb_model = xgb.XGBRegressor()
    print("Unpickling existing XGBRegressor model...")
    xgb_model.load_model(xgbr_model_file)
    
    # xgb_model.get_xgb_params()
    print("Loaded model :\n", xgb_model)
    print("with type\n", type(xgb_model))
    
# Check if there is any difference in the model saved and loaded predictions
# There are differences in predictions from loaded model
xgbr_predictions_new = xgb_model.predict(X_test[train_features])
pred_diff = np.setdiff1d(xgbr_predictions, xgbr_predictions_new)

len(pred_diff), xgbr_predictions.shape

In [ ]:
#### If XGBRegressor model exists, load it
retrieve_xgbr_file = './XGBRegressor_model_pickle'

if os.path.exists(retrieve_xgbr_file):
    # xgb_Classifier.save_model and load_model give an "le" error when trying to obtain score
    # Unpickling saved binary file if exist so that training do not need to done
    loaded_XGBRegressor = load_model_from_picklefile(retrieve_xgbr_file)
    print("Unpickling existing XGBRegressor model...")
    print("Loaded model :\n", loaded_XGBRegressor)
    print("with type\n", type(loaded_XGBRegressor))
    
# Must test with load_model()
xgbr_predictions_new = xgb_model.predict(X_test[train_features])
pred_diff = np.setdiff1d(xgbr_predictions, xgbr_predictions_new)

len(pred_diff),  xgbr_predictions.shape

# (C) Light Gradient Boosting Machine (LGBM)

- #### Exclude interim features to avoid strongly correlated features

In [ ]:
# len(feature_importance_df)
# feature_importance_df[["feature", "importance"]].sort_values(by="importance", ascending=False)[570:580]

# remove features to get better CV RMSE scores
print("No. of train_features :", len(train_features))
# features_to_remove = ['new_merchant_merchants_category_1_min_min', 'new_merchant_merchants_category_1_min_max',
#                       'new_merchant_merchants_category_1_max_min', 'new_merchant_merchants_category_1_max_max']

# 'new_merchant_merchants_active_months_lag12_min_min', 'new_merchant_merchants_avg_sales_lag3_max_min' was left out without a comma
# gives a better PB : 3.69303
# 'new_merchant_merchants_active_months_lag12_sum_min', 'new_merchant_merchants_active_months_lag12_min_min',
# 'new_merchant_merchants_avg_sales_lag3_max_min', 'new_merchant_merchants_avg_sales_lag6_max_max',
# 'new_merchant_merchants_avg_sales_lag6_max_min', 'new_merchant_merchants_active_months_lag6_sum_mean',
# 'new_merchant_merchants_avg_sales_lag12_max_max', 'new_merchant_merchants_avg_sales_lag3_max_max'

# ['new_merchant_merchants_merchant_category_id_std_min', 'new_merchant_merchants_merchant_category_id_std_std',
#  'new_merchant_merchants_merchant_category_id_std_max',
#  'hist_purchase_date_max_month', 'hist_purchase_date_min_month',
#  'new_merchant_purchase_date_max_month', 'new_merchant_purchase_date_min_month']

# 'hist_purchase_date_dayofweek_nunique',

# Exclude 'hist_category_1', 'hist_purchase_date_int', 'hist_card_purchase_date_max'
# 'new_card_purchase_date_max', 'new_purchase_date_int'
# ['hist_purchase_amount_max', 'hist_category_3_sum_purchase_amount_mean'] from list_discarded

# features_to_remove = ['hist_sum_category_1_1_purchase_amount','hist_purchase_date_year_nunique',
#                       'new_merchant_purchase_date_year_nunique',
                      
#                       'new_merchant_card_id_size', 
                      
#                       # Did not give better rank with lesser features (148 to 143)
#                       'hist_purchase_date_int', # removed 'new_purchase_date_int' in list_discarded
#                       'new_merchant_purchase_date_month_nunique', 'new_merchant_purchase_date_year_mean',
#                       'new_merchant_purchase_date_dayofweek_nunique']

# Remove features with low importance makes ranking worst than just removing KS list_discarded
features_to_remove = ['hist_sum_category_1_1_purchase_amount','hist_card_mean_purchase_amount_new',
                      'new_merchant_card_min_purchase_amount_new', 'hist_card_sum_purchase_amount_new',
                      'hist_card_min_purchase_amount_new', 'hist_category_1',
                      'new_merchant_card_sum_purchase_amount_new', 'new_merchant_purchase_amount_new',
                      'hist_sum_category_1_purchase_amount', 'new_merchant_card_median_purchase_amount_new']#'new_merchant_merchants_merchant_category_id_std_min',
                      #'new_merchant_merchants_merchant_category_id_std_std',
                      #'new_merchant_merchants_merchant_category_id_std_max',
                      # Present in 'X_train_137_KSfeats.csv' and was identified to be discarded
                      #'hist_w2v_hist_merchant_id_1_std']

train_features = list(set(train_features) - set(features_to_remove))

print("No. of train_features after removing some low important features :", len(train_features))

len(set(features_to_remove)), len(set(train_features))

In [ ]:
# Reference:
# old_list_discarded = ['hist_purchase_amount_max', 'hist_category_3_sum_purchase_amount_mean', 'hist_installments_mean']
# current_list_discarded = ['hist_purchase_amount_max', 'hist_category_3_sum_purchase_amount_mean',
#                           'hist_w2v_hist_merchant_id_0_std', 'hist_installments_mean']

## Using LGBM hyperparameters after tuning

In [ ]:
import lightgbm as lgb

# StratifiedKFold for binary or multiclass
# from sklearn.model_selection import StratifiedKFold
# import warnings
# warnings.filterwarnings('ignore')
np.random.seed(4590)

# Optimised lgbm params : improvement from fine tuning
lgbm_params_opt1 = {'num_leaves': 127,
                    'min_data_in_leaf': 148, # 'min_child_samples': 89 ignored
                    'objective':'regression',
                    'max_depth': 10,
                    'learning_rate': 0.005,
                    "boosting": "gbdt",
                    "feature_fraction": 0.7202,
                    "bagging_freq": 1,
                    "bagging_fraction": 0.8125,
                    "bagging_seed": 11,
                    "metric": 'rmse',
                    "lambda_l1": 0.3468,
                    "random_state": 4590, # 133,
                    "verbosity": -1}

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import KFold
np.random.seed(4590)

# Optimised lgbm params : check if there is improvement
# Version 41 (177 features) with the following hyperparameter tuning - 
lgbm_params_opt2 = {'num_leaves':62,
                    'min_data_in_leaf': 20,
                    'objective':'regression',
                    'max_depth': 8,
                    'learning_rate': 0.001,
                    # "min_child_samples": 20,
                    "boosting": "gbdt",
                    "feature_fraction": 0.9,
                    "bagging_freq": 1,
                    "bagging_fraction": 0.9,
                    "bagging_seed": 11,
                    "metric": 'rmse',
                    "lambda_l1": 0.1,
                    "verbosity": -1,
                    # "nthread": 4,
                    "random_state": 4590}

In [ ]:
# X[['feature_1', 'feature_2', 'feature_3']].head(2)
X[train_features].shape

In [ ]:
%%time
import time
categorical_feats = ['feature_1', 'feature_2', 'feature_3'] # 'feature_1'
num_round = 10000
n_splits= 7 # 5

# folds = KFold(n_splits=17, shuffle=True, random_state=15)
folds = KFold(n_splits=n_splits, shuffle=True, random_state=4590)

oof = np.zeros(len(X[train_features]))
lgbm_predictions = np.zeros(len(X_test[train_features]))
start = time.time()
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X[train_features].values, y.values)):
    print("LGBM fold n°{}".format(fold_))
    print("-" * 80)
    
    trn_data = lgb.Dataset(X.iloc[trn_idx][train_features],
                           label=y.iloc[trn_idx],
                           categorical_feature=categorical_feats)
    val_data = lgb.Dataset(X.iloc[val_idx][train_features],
                           label=y.iloc[val_idx],
                           categorical_feature=categorical_feats)

    # num_round = 10000
    lgbm_reg = lgb.train(lgbm_params_opt2, # lgbm_params
                         trn_data,
                         num_round,
                         valid_sets = [trn_data, val_data],
                         verbose_eval=100,
                         early_stopping_rounds = 200)
    
    oof[val_idx] = lgbm_reg.predict(X.iloc[val_idx][train_features], num_iteration=lgbm_reg.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = train_features
    fold_importance_df["importance"] = lgbm_reg.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    lgbm_predictions += lgbm_reg.predict(X_test[train_features], num_iteration=lgbm_reg.best_iteration) / folds.n_splits
    print()
    
    # Save model
    # lgbm_reg.save_model('lgbm_model.txt', num_iteration=bst.best_iteration)

print("CV RMSE score: {:<8.5f}".format(mean_squared_error(oof, y)**0.5))

X[train_features].shape

In [ ]:
lgbm_reg.save_model('lgbm_optmodel2_130KSfeats_K5_3.68718.txt', num_iteration=lgbm_reg.best_iteration)

## Observing feature importances
- Feature selection

In [ ]:
# Top features with max importance values
top_most_impt_features = (feature_importance_df[["feature", "importance"]]
                          .groupby("feature")
                          .max()
                          .sort_values(by="importance", ascending=False)[:21]).reset_index(drop=False)

# top_most_impt_features['feature']
top_most_impt_features

In [ ]:
cols = (feature_importance_df[["feature", "importance"]]
        .groupby("feature")
        .mean()
        .sort_values(by="importance", ascending=False).index) # [:1000].index for top 1000 features

best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]

plt.figure(figsize=(14,25))
sns.barplot(x="importance",
            y="feature",
            data=best_features.sort_values(by="importance", ascending=False))

plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.savefig('lgbm_opt_130KS_feats_K5_importances.png')

In [ ]:
# %%time
# features_correlation = pd.DataFrame(top_most_impt_features, columns=top_most_impt_features['feature'])

# colormap = plt.cm.RdBu
# plt.figure(figsize=(10,8))
# plt.title('Pearson Correlation of train features', y=1.05, size=15)
# sns.heatmap(features_correlation.astype(float).corr(),linewidths=0.1,vmax=1.0, 
#             square=True, cmap=colormap, linecolor='white', annot=True)

# plt.savefig('200feats_Correlation.png')

In [ ]:
# best_features.sort_values(by="importance", ascending=False).head(10)

In [ ]:
lgbm_predictions.shape

In [ ]:
# CV RMSE score: 3.64582, 3.64718, 3.64517, 3.64480, Best_3.5426
create_file_for_submission("lgbm_opt_130KS_merid_puramt_date_histmer_rdiff_w2v_installments_K5.csv", card_ids, lgbm_predictions)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
# X_train.drop(columns=features_to_remove, axis=1, inplace=True)
# X_test.drop(columns=features_to_remove, axis=1, inplace=True)
# X_train.shape, X_test.shape

In [ ]:
# loaded_train_feats = pd.read_csv('../input/test-predictions/X_train_137_KSfeats.csv')
# loaded_train_feats.shape

# excluded_features, list_discarded

# col_difference = set(loaded_train_feats.columns).symmetric_difference(X[train_features].columns)
# print("Difference in train and test features: {}".format(len(col_difference)))
# col_difference

# Difference in train and test features: 6
# {'card_id', 'first_active_month', 'hist_category_3_purchase_amount_sum_mean',
#  'hist_installments_mean', 'hist_purchase_amount_max', 'target'}

In [ ]:
X_train.to_csv('X_train_137_KSfeats.csv', index=False)
X_test.to_csv('X_test_136_KSfeats.csv', index=False)

In [ ]:
from IPython.display import FileLink

FileLink(r'lgbm_85_feats_catid_median_KFold17.csv')

## Progressions :
> RandomForestRegressor with GridSearchCV using 3 features</br>
> Rank : 3719 out of 4110 [Private Score : 3.81301]</br>
> Rank : 3749 out of 4110 [Public Score : 3.93004]

> RandomForestRegressor(max_features=2, n_estimators=30) with some historical features</br>
> Rank : 3416 out of 4110 [Private Score : 3.79375]</br>
> Rank : 3238 out of 4110 [Public Score : 3.87633]

> XGBRegressor with 45 historical and new merchant features with RMSE of 3.7006.</br>
> validation_0-rmse:1.78531	validation_1-rmse:3.75632</br>
> Rank : 2757 out of 4110 [Private Score : 3.65051] (67.08%)</br>
> Rank : 2824 out of 4110 [Public Score : 3.74154] (68.71%)

> XGBRegressor with 59 historical and new merchant features with RMSE of 3.6837.</br>
> validation_0-rmse:1.54589	validation_1-rmse:3.73675</br>
> Rank : 2755 out of 4110 [Private Score : 3.65021] (67.03%)</br>
> Rank : 2818 out of 4110 [Public Score : 3.74044] (68.56%)

> XGBRegressor with 69 historical and new merchant features with RMSE of 3.6837.</br>
> validation_0-rmse:1.60083	validation_1-rmse:3.73686</br>
> Final RMSE :  3.686220904191716</br>
> Rank : 2756 out of 4110 [Private Score : 3.65033] (67.06%)</br>
> Rank : 2781 out of 4110 [Public Score : 3.73777] (67.66%)

> XGBRegressor with 79 historical (10 OHE for features1-3) and new merchant features with RMSE of 3.7027.</br>
> validation_0-rmse:1.51994	validation_1-rmse:3.73630</br>
> Final RMSE :  3.6854681990466274</br>
> Rank : 2747 out of 4110 [Private Score : 3.64916] (66.84%)</br>
> Rank : 2756 out of 4110 [Public Score : 3.73545] (67.06%)</br>

> LGBM with 79 historical (10 OHE for features1-3) and new merchant features with RMSE of 3.6545.</br>
> [1041] training's rmse: 3.49544	valid_1's rmse: 3.60891
> CV score: 3.65450 </br>
> Rank : 2115 out of 4110 [Private Score : 3.62122] (51.46%)</br>
> Rank : 2370 out of 4110 [Public Score : 3.70550] (57.66%)</br>

> LGBM with 83 (10 OHE for features1-3 and new merchant purchase date min max mode) features at RMSE of 3.65311.</br>
> [1120] training's rmse: 3.4844	valid_1's rmse: 3.6065
> CV score: 3.65311 </br>
> Rank : 1947 out of 4110 [Private Score : 3.62025] (47.37%)</br>
> Rank : 2296 out of 4110 [Public Score : 3.70437] (55.86%)</br>

> LGBM with 87 using median for nan (new merchant purchase date min max mode with merchant data) features.</br>
> [1102] training's rmse: 3.47996	valid_1's rmse: 3.60637
> CV RMSE score: 3.65275</br>
> Rank : 1498 out of 4110 [Private Score : 3.61882] (36.45%)</br>
> Rank : 2227 out of 4110 [Public Score : 3.70239] (54.18%)</br>

> LGBM_KFold7 with 95_OHE using median for nan (new merchant purchase date min max mode with merchant data) features.</br>
> Rank : 1410 out of 4110 [Private Score : 3.61813] (34.31%)</br>
> Rank : 2185 out of 4110 [Public Score :  3.70141] (53.16%)</br>
> CV RMSE score: 3.65106

> LGBM_KFold17 with 128 features using median for nan (new merchant purchase date min max mode with merchant data).</br>
> Rank : 1805 out of 4110 [Private Score : 3.61959] (43.92%)</br>
> Rank : 2184 out of 4110 [Public Score :  3.70135] (53.14%)</br>
> CV RMSE score: 3.64977

> LGBM_KFold17 with 136 features using median for nan (new merchant purchase date min max mode with merchant data).</br>
> Rank : 1655 out of 4110 [Private Score : 3.61937] (40.27%)</br>
> Rank : 2160 out of 4110 [Public Score :  3.70097] (52.55%)</br>
> CV RMSE score: 3.64882

> LGBM_KFold17_merchantid with 137 features using median for nan (new merchant purchase date min max mode with merchant data, purchase date max ratio) at CV RMSE of 3.64613.</br>
> Rank : 963 out of 4110 [Private Score : 3.61572] (23.43%)</br>
> Rank : 1722 out of 4110 [Public Score :  3.69594] (41.90%)</br>
> CV RMSE score: 3.64613

> LGBM_KFold17_merchantid with 137 features using median for nan (new merchant purchase date min max mode with merchant data, purchase date max ratio, pd_max) at CV RMSE of 3.64625.</br>
> Rank : 922 out of 4110 [Private Score : 3.61546] (22.43%)</br>
> Rank : 1672 out of 4110 [Public Score :  3.69539] (40.68%)</br>
> CV RMSE score: 3.64625
> Early stopping, best iteration is:
> [1152]	training's rmse: 3.47992	valid_1's rmse: 3.43485

> LGBM_KFold17_merchantid with 142 features using median for nan (new merchant purchase date min max mode with merchant data, 'new_hist_purchase_date_max_ratio', pd_max, 'new_hist_purchase_amount_max_ratio') at CV RMSE of 3.64680.</br>
> **Rank : 352 out of 4110 [Private Score : 3.61319] (8.56%)**</br>
> Rank : 1560 out of 4110 [Public Score :  3.69400] (37.96%)</br>
> CV RMSE score: 3.64680

> LGBM_KFold17_merchantid with 138 features using median for nan (new merchant purchase date min max mode with merchant data, 'new_hist_purchase_date_max_ratio', pd_max, 'new_hist_purchase_amount_max_ratio') at CV RMSE of 3.64599.</br>
> Rank : 356 out of 4110 [Private Score : 3.61322] (8.66%)</br>
> Rank : 1525 out of 4110 [Public Score :  3.69357] (37.10%)</br>
> CV RMSE score: 3.64599</br>
> Wall time: 37min 5s

> LGBM_KFold17_merchantid with 139 features using median for nan (new merchant purchase date min max mode with merchant data, 'new_hist_purchase_date_max_ratio', pd_max, 'new_hist_purchase_amount_max_ratio', 'hist_category_1_purchase_amount_sum_mean).</br>
> Rank : 405 out of 4110 [Private Score : 3.61363] (9.85%)</br>
> Rank : 1504 out of 4110 [Public Score :  3.69340] (36.59%)</br>
> CV RMSE score: 3.64582</br>
> Wall time: 36min 25s

> LGBM_KFold17_merchantid with 142 features using median for nan (new merchant purchase date min max mode with merchant data, 'new_hist_purchase_date_max_ratio', pd_max, 'new_hist_purchase_amount_max_ratio',  'new_hist_purchase_date_min_ratio', 'hist_category_1_purchase_amount_sum_mean', 'new_merchant_category_1_purchase_amount_sum_mean', ratio and diff) at CV RMSE of 3.64659.
> Rank : 384 out of 4110 [Private Score : 3.61342] (9.34%)</br>
> Rank : 1447 out of 4110 [Public Score : 3.69303] (35.21%)</br>
> CV RMSE score: 3.64659</br>
> Wall time: 37min 26s</br>

> LGBM_KFold17_merchantid with 142 features using median for nan (new merchant purchase date min max mode with merchant data, 'new_hist_purchase_date_max_ratio', pd_max, 'new_hist_purchase_amount_max_ratio',  'new_hist_purchase_date_min_ratio', 'hist_category_1_purchase_amount_sum_mean', 'new_merchant_category_1_purchase_amount_sum_mean', ratio and diff) at CV RMSE of 3.64659.
> Rank : 795 out of 4110 [Private Score : 3.61475] (19.34%)</br>
> Rank : 1253 out of 4110 [Public Score : 3.69242] (30.49)</br>
> CV RMSE score: 3.64595 </br>
> Wall time: 36min 42s</br>

> LGBM_KFold17_merchantid with 150 features (new merchant purchase date min max mode with merchant data, 'new_hist_purchase_date_max_ratio', pd_max, 'new_hist_purchase_amount_max_ratio',  'new_hist_purchase_date_min_ratio', 'hist_category_1_purchase_amount_sum_mean', 'new_merchant_category_1_purchase_amount_sum_mean', ratio and diff, 'hist_w2v_hist_merchant_cat_id_std').
> Rank : 687 out of 4110 [Private Score : 3.61394] (16.72%)</br>
> Rank : 1236 out of 4110 [Public Score : 3.69227] (30.07%)</br>
> CV RMSE score: 3.64573</br>
> Wall time: 37min 46s</br>

> LGBM_KFold17_merchantid with 162 features (new merchant purchase date min max mode with merchant data, 'new_hist_purchase_date_max_ratio', pd_max, 'new_hist_purchase_amount_max_ratio',  'new_hist_purchase_date_min_ratio', 'hist_category_1_purchase_amount_sum_mean', 'new_merchant_category_1_purchase_amount_sum_mean', 'w2v_hist_merchant_category_id_0-4_std', 'w2v_hist_merchant_id_0-4_std', 'w2v_new_merchant_category_id_0-4_std', 'w2v_new_merchant_id_0-4_std' ratio and diff interactions).</br>
> Rank : 955 out of 4110 [Private Score : 3.61569] (23.24%)</br>
> Rank : 1066 out of 4110 [Public Score : 3.69122] (25.94%)</br>
> CV RMSE score: 3.64517</br>
> Wall time: 40min 32s</br>

> LGBM_KFold17_merchantid with 162 features (new merchant purchase date min max mode with merchant data, 'new_hist_purchase_date_max_ratio', pd_max, 'new_hist_purchase_amount_max_ratio',  'new_hist_purchase_date_min_ratio', 'hist_category_1_purchase_amount_sum_mean', 'new_merchant_category_1_purchase_amount_sum_mean', 'w2v_hist_merchant_category_id_0-4_std', 'w2v_hist_merchant_id_0-4_std', 'w2v_new_merchant_category_id_0-4_std', 'w2v_new_merchant_id_0-4_std' ratio and diff interactions).</br>
> Rank : 908 out of 4110 [Private Score : 3.61537] (22.09%)</br>
> Rank : 1020 out of 4110 [Public Score : 3.69069] (24.82%)</br>
> CV RMSE score: 3.64634</br>
> Wall time: 47min 6s</br>

> LGBM_KFold17_merchantid with 130 or 133 KS_0.01features (new merchant purchase date min max mode with merchant data, 'new_hist_purchase_date_max_ratio', pd_max, 'new_hist_purchase_amount_max_ratio',  'new_hist_purchase_date_min_ratio', 'hist_category_1_purchase_amount_sum_mean', 'new_merchant_category_1_purchase_amount_sum_mean', 'w2v_hist_merchant_category_id_0-4_std', 'w2v_hist_merchant_id_0-4_std', 'w2v_new_merchant_category_id_0-4_std', 'w2v_new_merchant_id_0-4_std', ratio and diff interactions, installments_outliers_median).</br>
> Rank : 990 out of 4110 [Private Score : 3.61597] (24.09%)</br>
> Rank : 920 out of 4110 [Public Score : 3.68906] (22.38%)</br>
> CV RMSE score: 3.64537</br>
> Wall time: 31min 40s</br>

> LGBM_KFold17_merchantid_opt with 130 KS_0.01features (new merchant purchase date min max mode with merchant data, 'new_hist_purchase_date_max_ratio', pd_max, 'new_hist_purchase_amount_max_ratio',  'new_hist_purchase_date_min_ratio', 'hist_category_1_purchase_amount_sum_mean', 'new_merchant_category_1_purchase_amount_sum_mean', 'w2v_hist_merchant_category_id_0-4_std', 'w2v_hist_merchant_id_0-4_std', 'w2v_new_merchant_category_id_0-4_std', 'w2v_new_merchant_id_0-4_std', ratio and diff interactions, installments_outliers_median).</br>
> Rank : 948 out of 4110 [Private Score : 3.61562] (23.07%)</br>
> Rank : 869 out of 4110 [Public Score : 3.68811] (21.14%)</br>
> CV RMSE score: 3.64531</br>
> Wall time: 32min 36s</br>

> LGBM_KFold5_merchantid_opt2 with 130 KS_0.01features (new merchant purchase date min max mode with merchant data, 'new_hist_purchase_date_max_ratio', pd_max, 'new_hist_purchase_amount_max_ratio',  'new_hist_purchase_date_min_ratio', 'hist_category_1_purchase_amount_sum_mean', 'new_merchant_category_1_purchase_amount_sum_mean', 'w2v_hist_merchant_category_id_0-4_std', 'w2v_hist_merchant_id_0-4_std', 'w2v_new_merchant_category_id_0-4_std', 'w2v_new_merchant_id_0-4_std', ratio and diff interactions, installments_outliers_median).</br>
> (5 KFold performs better than 17 KFold)</br>
> Rank : 1415 out of 4110 [Private Score : 3.61817] (34.43%)</br>
> **Rank : 495 out of 4110 [Public Score : 3.68718] (12.04%)**</br>
> CV RMSE score: 3.65024</br>
> Wall time: 8min 31s</br>
Early stopping, best iteration is:
[1233]	training's rmse: 3.38217	valid_1's rmse: 3.63805

--
Early stopping, best iteration is:
[1164]	training's rmse: 3.41147	valid_1's rmse: 3.4898
--
Early stopping, best iteration is:
[826]	training's rmse: 3.49935	valid_1's rmse: 3.49333

--
Early stopping, best iteration is:
[1892]	training's rmse: 3.38284	valid_1's rmse: 3.48561
CV RMSE score: 3.64479 
--
Early stopping, best iteration is:
[902]	training's rmse: 3.48923	valid_1's rmse: 3.48783
CV RMSE score: 3.64541
--

--
> Remove the 'new_merchant_purchase_date_max' and 'hist_purchase_date_max' after creating the ratio
> Early stopping, best iteration is:
> [1007]	training's rmse: 3.49581	valid_1's rmse: 3.43503
> CV RMSE score: 3.64655 
> Wall time: 52min 27s

# (D) Feature Extractions (with Sequential Feed-forward Neural Network), Deep Learning
- #### Train our model</br>
> For training a model, we use the fit function, which trains the model for a given number of epochs, which refers to the number of times we pass our dataset through our model to train it.  We use callbacks.TensorBoard to writes a log for TensorBoard, which allows you to visualize dynamic graphs of your training and test metrics, as well as activation histograms for the different layers in your model.  To save our model after every epoch we using callbacks.ModelCheckpoint for it.

- #### Import libraries for Deep Learning

In [ ]:
import keras
from keras import callbacks
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import models, layers
# from keras.optimizers import RMSprop, SGD
from keras.optimizers import Adam, Adadelta
from keras.optimizers.schedules import ExponentialDecay
from keras.metrics import RootMeanSquaredError

# For loading saved model in h5 format
from keras.models import load_model

#As shown:
#1) No convolution layer.
#2) Make the input shape equals to your regression features numbers.
#3) The network ends with output layer with only 1 output and no activation (it will be a linear layer, applying an activation function would constrain the range the output can take)
#4) Compile the network with the mse loss function—mean squared error, the square of the difference between the predictions and the targets. This is a widely used loss function for regression problems.
#5) Make the metric to monitor during training as mean absolute error (MAE). It’s the absolute value of the difference between the predictions and the targets. 

In [ ]:
# tensorboard = callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True)
model_checkpoints = callbacks.ModelCheckpoint("weights_{epoch:02d}_{val_loss:.2f}.h5", monitor='val_loss',
                                              verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [ ]:
def build_model(input_df):
    print("No. of features :", input_df.shape[1])
    model = models.Sequential()
    model.add(layers.Dense(512, activation='relu', input_shape=(input_df.shape[1],))) # 1024
    model.add(layers.Dropout(0.2)),
    model.add(layers.Dense(256, activation='relu')) # 512
    model.add(layers.Dropout(0.3)),
    model.add(layers.Dense(128, activation='relu')) # 128 
    model.add(layers.Dropout(0.5)),
    model.add(layers.Dense(1))

    lr_schedule = ExponentialDecay(initial_learning_rate=1e-3,
                                   decay_steps=10000, decay_rate=0.9)
    
    model.compile(optimizer=Adam(learning_rate=1e-3), # Adadelta(learning_rate=0.001, rho=0.95, epsilon=1e-07, name="Adadelta")
                  loss='mean_squared_error', # mse
                  metrics=['RootMeanSquaredError'])
    return model

In [ ]:
# import keras
# from keras import callbacks
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, BatchNormalization
# from tensorflow.keras import models, layers
# # from keras.optimizers import RMSprop, SGD
# from keras.optimizers import Adam, Adadelta
# from keras.optimizers.schedules import ExponentialDecay
# from keras.metrics import RootMeanSquaredError

# def build_model(input_df):
#     print("No. of features :", input_df.shape[1])
#     model = models.Sequential()

#     model.add(Dense(2 ** 10, input_dim = input_df.shape[1],
#                     kernel_initializer='glorot_uniform', activation='relu')) # init='random_uniform',
#     model.add(Dropout(0.25))    
#     model.add(BatchNormalization())
#     model.add(Dense(2 ** 9, activation='relu')) # kernel_initializer='random_uniform'
#     model.add(BatchNormalization())
#     model.add(Dropout(0.25)) 
#     model.add(Dense(2 ** 5, activation='relu')) # kernel_initializer='random_uniform'
#     model.add(BatchNormalization())
#     model.add(Dropout(0.25))      
#     model.add(Dense(1))

#     #     model.compile(loss='mean_squared_error', optimizer='adam')
#     model.compile(optimizer=Adam(learning_rate=1e-3), # Adadelta(learning_rate=0.001, rho=0.95, epsilon=1e-07, name="Adadelta")
#                   loss='mse',
#                   metrics=['RootMeanSquaredError'])
#     return model

In [ ]:
X[train_features].shape, X_test[train_features].shape

In [ ]:
dl_model = build_model(X[train_features])

dl_model.summary()

In [ ]:
%%time
model_log = dl_model.fit(X[train_features], y, epochs=20, batch_size=64, # 16
                         validation_split = 0.2,
                         callbacks=[model_checkpoints]) # , verbose=0)

# Beat this, val_root_mean_squared_error: 3.9318

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2,figsize=(15,5))

ax1.plot(model_log.history['root_mean_squared_error'])
ax1.plot(model_log.history['val_root_mean_squared_error'])
ax1.set_title('Root Mean Squared Error (RMSE)')
ax1.set(xlabel='Epoch', ylabel='RMSE')
ax1.legend(['train', 'validation'], loc='upper right')
ax1.xaxis.set_major_locator(MaxNLocator(integer=True))

ax2.plot(model_log.history['loss'])
ax2.plot(model_log.history['val_loss'])
ax2.set_title('Loss (Lower Better)')
ax2.set(xlabel='Epoch', ylabel='Loss')
ax2.legend(['train', 'validation'], loc='upper right')
ax2.xaxis.set_major_locator(MaxNLocator(integer=True))

plt.savefig('130KSfeats_DL3_train_validation.png')

In [ ]:
# first_batchdata = X[train_features].iloc[:5]
# first_batchlabels = y.iloc[:5]

# print("XBGR Predictions: ", xgb_model.predict(first_batchdata))
# print("LGBM Predictions: ", lbg_reg.predict(first_batchdata))
# print("DL Predictions: ", dl_model.predict(first_batchdata))

# print("Labels: ", list(first_batchlabels))

# test_mse_score, test_rmse_score = dl_model.evaluate(first_batchdata, first_batchlabels)

# print(f'test_mse_score : {test_mse_score}')
# print(f'test_rmse_score : {test_rmse_score}')

# # The attribute model.metrics_names will give you the display labels for the evalution outputs.
# print("\nModel metrics :", dl_model.metrics_names)

In [ ]:
X_test[train_features].shape

In [ ]:
dl_predictions = dl_model.predict(X_test[train_features])
dl_predictions.shape, X_test[train_features].shape

# DL_138 feats with Pri_score : 3.83928, Pub_score : 3.95739
# DL_130 KSfeats with Pri_score : 3.84114, Pub_score : 3.95931
# DL3_v1_130 KSfeats with Pri_score : 3.82566, Pub_score : 3.94311

In [ ]:
# https://keras.io/api/models/model_saving_apis/
dl_model.save('DL3_v1_130KSfeat_adam.h5')

# deletes the existing model
# del dl_model

In [ ]:
create_file_for_submission("DL3_v1_130KSfeat_adam.csv", card_ids, dl_predictions[:, 0])

# (E) Blending prediction results from different models (heterogeneous regressors) by averaging

In [ ]:
!ls -tl ../input/test-predictions

In [ ]:
!ls

- ## Heterogeneous regressors predictions

In [ ]:
# Load from saved predictions from different models
# '../input/testpredictions/lgbm_162_feats_KFold17.csv'
lgbm_1_pred = pd.read_csv('lgbm_opt2_130KS_merid_puramt_date_histmer_rdiff_w2v_installments_K5.csv')
lgbm_2_pred = pd.read_csv('../input/test-predictions/lgbm_130KSfeats_K17.csv')
lgbm_3_pred = pd.read_csv('../input/test-predictions/lgbm_141_feats.csv')
xgbr_1_pred = pd.read_csv('../input/test-predictions/xgbr_126KS_feats_KFold17.csv')
xgbr_2_pred = pd.read_csv('../input/test-predictions/xgbr_141_feats.csv')
rf_pred = pd.read_csv('../input/test-predictions/rf_130KS_feats.csv')
dl_pred = pd.read_csv('DL3_v1_130KSfeat_adam.csv') # dl3_130KS_feats.csv'

concat_test_pred = pd.concat([lgbm_1_pred['target'], lgbm_2_pred['target'], lgbm_3_pred['target'],
                              xgbr_1_pred['target'], xgbr_2_pred['target'],
                              rf_pred['target'], dl_pred['target']],
                             axis=1, keys=['LGBM_1_Pred', 'LGBM_2_Pred', 'LGBM_3_Pred',
                                           'XGBR_1_Pred', 'XGBR_2_Pred',
                                           'RF_Pred', 'DL_Pred'])

# dl_preds = pd.read_csv('../input/testpredictions/xgbr_lgbm_dl_79feat.csv')
# concat_pred_df = pd.concat([xgbr_preds['target'], lgbm_preds['target'], dl_preds['target']],
#                             axis=1, keys=['XGBR_Pred', 'LGBM_Pred', 'DL_Pred'])

# concat_pred_df['Avg_Pred'] = concat_pred_df.mean(axis=1)

# xgbr_preds.loc[:5, 'target'], lgbm_preds.loc[:5, 'target'], dl_preds.loc[:5, 'target']

concat_test_pred

In [ ]:
# xgbr_predictions_df = pd.DataFrame(xgbr_predictions, columns = ['XGBR_Pred'])
# lgbm_predictions_df = pd.DataFrame(lgbm_predictions, columns = ['LGBM_Pred'])
# dtreg_predictions_df = pd.DataFrame(tree_reg_predictions, columns = ['DTR_Pred'])
# rfb_predictions_df = pd.DataFrame(rf_predictions, columns = ['RFB_Pred'])
# dl_predictions_df = pd.DataFrame(dl_predictions[:, 0], columns = ['DL_Pred'])

# concat_pred_df = pd.concat([xgbr_predictions_df, lgbm_predictions_df, dtreg_predictions_df], axis=1, sort=False)
# concat_pred_df = pd.concat([concat_pred_df, rfb_predictions_df], axis=1, sort=False)
# concat_pred_df = pd.concat([concat_pred_df, dl_predictions_df], axis=1, sort=False)

# concat_pred_df

In [ ]:
concat_test_pred['Avg_Pred'] = concat_test_pred.mean(axis=1)
concat_test_pred

In [ ]:
X[train_features].shape

In [ ]:
# Average predicted values from XGBR, LGBM, RF and DL
create_file_for_submission("lgbm_xgbr_rfb_dl3_130feat.csv", card_ids, concat_test_pred['Avg_Pred'])

# (F) Postprocessing

In [ ]:
predicted_loyality_score = concat_test_pred['LGBM_1_Pred']
ax = predicted_loyality_score.plot.hist(bins=20, figsize=(6, 5))
_ = ax.set_title("Predicted target histogram")
plt.show()

fig, axs = plt.subplots(1,2, figsize=(12, 5))
_ = predicted_loyality_score[predicted_loyality_score > 10].plot.hist(ax=axs[0])
_ = axs[0].set_title("Predicted target histogram for values greater than 10")
_ = predicted_loyality_score[predicted_loyality_score < -10].plot.hist(ax=axs[1])
_ = axs[1].set_title("Predicted target histogram for values less than -10")
plt.show()

In [ ]:
lgbm_pred = lgbm_1_pred
lgbm_pred.loc[lgbm_pred['target'] < -10]

In [ ]:
# Predicted 'target' less than -10 is considered outliers
card_id_lt_neg10 = lgbm_pred.loc[lgbm_pred['target'] < -10, 'card_id']
len(list(card_id_lt_neg10))

In [ ]:
# Post-processing by assigning cards with outlier 'target' value
for id in card_id_lt_neg10:
    lgbm_pred.loc[lgbm_pred['card_id']==id,'target'] = -33.21928095
    
lgbm_pred.loc[lgbm_pred['target'] < -10]

In [ ]:
# Average predicted values from LGBM, XGBR, RF and DL
create_file_for_submission("lgbm_130KSfeats_postprocess_.csv", card_ids, lgbm_pred['target'])

* # Conclusions :
> #### 1. Single LGBM with KS 130 features performs the best
> #### 2. Outlier predictions are important for this problem; thus the multiple model combinations like blending method (3) performs well.
> #### 3. With hyperparameter tunings and post processing can potentially improve on the ML models.
> #### 4. More feature engineering with KS could also improve the final model for the problem.

# Part 4 : Visualizations

In [ ]:
concat_test_pred.head(5)

- #### Comparisons on prediction results from different models

In [ ]:
no_of_samples = 50
lgbm_pred_samples = concat_test_pred.loc[:no_of_samples, 'LGBM_1_Pred']
xgbr_pred_samples = concat_test_pred.loc[:no_of_samples, 'XGBR_1_Pred']
rf_pred_samples = concat_test_pred.loc[:no_of_samples, 'RF_Pred']
dl_pred_samples = concat_test_pred.loc[:no_of_samples, 'DL_Pred']

# rfb_pred_samples = concat_pred_df.loc[:no_of_samples, 'RFB_Pred']
#dtreg_pred_samples = concat_pred_df.loc[:no_of_samples, 'DTR_Pred']

model_avg_pred_samples = concat_test_pred.loc[:no_of_samples, 'Avg_Pred']
# esm_reg_pred_samples = ensemble_reg_pred.loc[:no_of_samples, 'ESM_LGBM_Pred']

# https://matplotlib.org/stable/tutorials/colors/colors.html
# https://matplotlib.org/stable/api/markers_api.html

plt.figure(figsize=(20,12))
plt.plot(lgbm_pred_samples, 'b^', label='LGBM_1')
plt.plot(xgbr_pred_samples, 'gd', label='XGBR_1')
plt.plot(rf_pred_samples, 'ys', label='RFB')
plt.plot(dl_pred_samples, 'c+', label='DL3') # DL5
plt.plot(model_avg_pred_samples, 'r*', ms=10, label='Avg')

# plt.plot(esm_reg_pred_samples, 'm.', ms=10, label='ESM_LGBM')

plt.tick_params(axis='x', which='both', bottom=False, top=False,
                labelbottom=False)

plt.ylabel('predicted')
plt.xlabel('Test samples')
plt.legend(loc="best")
plt.title('Model predictions with their averages and stacking')


plt.savefig('Model_Prediction_plots.png')
plt.show()

## End of Program